# Kelompok 2: Avengers Team: E Commerce Churn Prediction 


# LOAD DATASET

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
# sklearn import for data pre-processing
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer, KNNImputer

# sklearn import for LogisticRegression and RandomForest algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# ignore warnings( not suggested for real-life projects)
import warnings
warnings.filterwarnings('ignore')


In [ ]:
from matplotlib import rcParams

rcParams['figure.figsize'] = 12, 6
rcParams['lines.linewidth'] = 3
rcParams['xtick.labelsize'] = 'x-large'
rcParams['ytick.labelsize'] = 'x-large'

### 1. Dari GDRIVE

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import pandas as pd
df = pd.read_excel('/content/gdrive/My Drive/FINAL PROJECT/E Commerce Dataset.xlsx', sheet_name = 'E Comm')

In [ ]:
df_stg1 = df.copy()
df_stg1.head()

### 2. Dengan Upload Dataset

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd
df = pd.read_excel('E Commerce Dataset.xlsx', sheet_name = 'E Comm')
df_stg1 = df.copy()
df_stg1.head()

### 3. Import dari direktori pc

In [ ]:
df = pd.read_excel('E Commerce Dataset.xlsx', sheet_name = 'E Comm')

In [ ]:
df_stg1 = df.copy()
df_stg1.head()

# Stage 1 - Exploratory Data Analysis

Import Libraries and Dataset first!

## CHURN 

In [ ]:
churn = df[df['Churn']==1]
jumlah_churn = churn['Churn'].sum()
jumlah_churn

Jumlah customer yang churn sebanyak 948 customer dari total 5630 customer

In [ ]:
total_customer = df.shape[0]
a = df.groupby(['Churn'])['CustomerID'].count().reset_index()
a['Persentase(%)'] = a['CustomerID']/total_customer*100
a

In [ ]:
A=['Churn', 'Tidak Churn']
T=[16.84, 83.16]
  
plt.pie(T, labels=A,autopct= '%1.1f%%')
plt.title('Churn Proportion', size = 24)
plt.show()

## <b> 1. Descriptive Statistics

informasi general tentang dataframe

In [ ]:
df_stg1.info()

In [ ]:
print('Data yang duplikat = ', df_stg1.duplicated().sum())

<b> Kolom yang memiliki Missing value

In [ ]:
df_stg1.isna().sum().sort_values(ascending=False)

<b> Pengamatan:
1. Data terdiri dari 5630 baris dengan total 20 kolom
2. Terdapat null/missing values pada kolom: `DaySinceLastOrder`,`OrderAmountHikeFromlastYear`,`Tenure`,`OrderCount`,`CouponUsed`,`HourSpendOnApp`,`WarehouseToHome`
3. Untuk tipe data churn, ada baiknya jika bertipe boolean. akan tetapi dalam case ini tidak terlalu mengganggu proses pengolahan data nantinya. Selain itu, sepertinya tidak ada issue yang mencolok pada tipe data untuk setiap kolom (<b> Semua tipe data sudah sesuai)
4. Tidak ada duplicate data antar baris.

### UNDERSTAND THE DATA
- Customer ID ---> Berisikan ID unik dari customer
- Churn ----> Churn atau tidak?
- Tenure ---> Berapa lama customer menggunakan layanan (loyalitas) (bulan)
- PreferredloginDevice ---> Customer login dengan device komputer, handphone dll
- CityTier ---> Tingkatan kota (1,2,3)
- WareHousetoHome ---> Jarak gudang (warehouse) e-commercenya ke rumah pelanggan
- PreferredPaymentMod ---> Metode pembayaran yang lebih dipilih untuk digunakan
- Gender ---> Jenis kelamin
- HourSpendOnApp ---> Waktu yang dihabiskan untuk menjelajah di aplikasi (dalam jam)
- Number of Device Registerd -----> Jumlah total perangkat yang terdaftar
- PreferedOrderCat ---> Kategori produk yang sering diorder
- SatisfactionScore ----> Tingkat kepuasan
- MaritalStatus ----> Status pernikahan
- NumberOfAddress  ---> Jumlah alamat yang ditambahkan customer 
- OrderAmountHikeFromlastYear -----> Kenaikan Jumlah Pesanan Dari Tahun Lalu
- Complain -----> Complain atau tidak?
- CouponUsed ----> Jumlah total Kupon yang telah digunakan bulan lalu
- OrderCount ----> Jumlah orderan bulan lalu
- DaySinceLastOrder -----> Jarak hari ini dengan hari terakhir/rentang waktu customer order pesanan. 
- CashbackAmount ----> Jumlah cashback bulan lalu

In [ ]:
df_stg1.describe().transpose()

In [ ]:
percentage_of_null_values = ((df_stg1.isnull().sum())*100/len(df)).sort_values(ascending = False)
percentage_of_null_values

<b> Untuk kolom/feature yang memiliki missing value, karena nilai null > 5% data, maka akan dilakukan imputation berdasarkan sebaran/distribusi datanya. Jika data skew ke kanan (mean>median), maka digunakan nilai median karena nilai median robust terhadap outlier. Atau opsi lain, menggunakan nilai IQR (Q3-Q1) yang juga robust terhadap outlier. 

#### <b> Data Snippet

In [ ]:
df_stg1.sample(5)

### STATISTICAL SUMMARY

#### <b> Grouping by Data Variable Type (Pick + Separate Columns)

In [ ]:
# pengelompokan kolom berdasarkan jenisnya
nums = ['Tenure','CityTier', 'WarehouseToHome', 'HourSpendOnApp', 'NumberOfDeviceRegistered','SatisfactionScore', 'DaySinceLastOrder','NumberOfAddress', 'OrderAmountHikeFromlastYear', 'CouponUsed', 'OrderCount', 'CashbackAmount']
cats = ['PreferredLoginDevice', 'PreferredPaymentMode', 'Gender', 'PreferedOrderCat', 'MaritalStatus']

<b> A. DATA NUMERICAL

In [ ]:
df_stg1[nums].describe().transpose()

Beberapa kolom/feature sudah cukup simetrik distribusinya (mean dan median tidak terlalu jauh), namun ada beberapa kolom yang  skew, dilihat dari nilai mean dan median ---->        "mean>median" = skew ke kanan (akan kita lihat visualisasi nya di bawah) yaitu kolom `Tenure`, `WareHouseToHome`, `OrderCount`,`DaySinceLastOrder`,`CashbackAmount`, `NumberOfAddress`, `OrderAmountHikeFromlastYear`  dst .

<b> B. DATA CATEGORICAL

In [ ]:
df_stg1[cats].describe().transpose()

In [ ]:
df[nums].info()

In [ ]:
df[cats].info()

#### <b> Value Counting

In [ ]:
for col in cats:
    print(f"Value count column {col}:")
    print(df[col].value_counts())
    print()

<b> Untuk Tipe data categorical, sekilas tidak ada yang aneh untuk descripstive statistic nya. Akan tetapi jika divisualisasikan (di Univariate analysis), akan terdapat beberapa kateogori yang sebenarnya 1 pengertian, tetapi dibuat dalam 2 definisi. contoh : CC dan credit card pada fitur metode pembayaran, COD dan Cash on Delivery pada kolom PreferredPaymentMode. selengkapnya akan dibahas di preprocessing. 

## <b> 2. Univariate Analysis

In [ ]:
df_stg1_1 = df_stg1.copy()

In [ ]:
df_stg1_1.drop(['CustomerID'], axis=1, inplace=True) #apakah kolom perlu di hapus disini? karna akan dijadikan target saat proses pemodelan?

In [ ]:
df_stg1_1.head()

In [ ]:
df_stg1_1.describe()

In [ ]:
plt.figure(figsize=(30,20))
features = nums
for i in range(0, len(features)):
    plt.subplot(3, 7, i+1)
    sns.boxplot(y=df_stg1[features[i]], color='salmon', orient='v')
    plt.xlabel(features[i])

Untuk boxplot, hal paling penting yang harus kita perhatikan adalah keberadaan outlier.
* Outlier terlihat utamanya pada kolom `CouponUsed`, `OrderCount` dan `CashbackAmount`
* Outlier yang tidak terlalu banyak ada pada kolom `Tenure`, `WareHouseToHome`, `TimeSpendOnApp`, `NumberOfDeviceRegistered`, `OrderAmountHikeFromLastYear`
* Dari boxplotnya juga tampak mana distribusi yang terlihat *skewed*: `Tenure`, `Couponused`, `OrderCount`                                       dan agak *skewed* --> `OrderAmountHikeFromlastYear` 
* Beberapa kolom sudah memiliki distribution data yang cukup baik/normal, seperti `HourSpendOnApp`, `NumberOfDeviceRegister`dan `SatisfactionScore`

<b> Outlier ini nantinya dapat di "Handle" dengan Dua opsi, yaitu 
1. Menghapus Outlier berdasarkan IQR 
2. Menghapus Outlier berdasarkan Z-Score (untuk outlier yang ekstrim ----> pada data dengan sebaran/distribusi normal, sekitar ~0,3%). 
disclaimer, metode IQR tidak disarankan karena dapat meghapus outlier dalam jumlah yang sangat ekstrim, yang bahkan dapat mengilangkan data > 5%. Oleh karena itu, dalam kasus ini digunakan metode Z-score saja dalam menangani outlier

In [ ]:
features = nums
plt.figure(figsize=(30, 20))
for i in range(0, len(features)):
    plt.subplot(3, 5, i+1)
    sns.kdeplot(x=df_stg1[features[i]], color='salmon')
    plt.xlabel(features[i])

grafik yang menarik dari data numeric adalah :
1. tenure : right skewed, ada outlier disisi kanan
2. CityTier : Normal (range 1-3). 
3. warehouse to home : right skewed, ada outlier disisi kanan
4. HourSpendOnapp : Normal, ada outlier di kiri dan kanan
5. NumberOfDeviceRegistered : Normal, ad outlier di kiri dan kanan
6. SatisfactionScore : Normal, tidak ada outlier (range 1-5)
7. days since last order : right skewed, ada outlier disisi kanan
8. number of address : right skewed, ada outlier disisi kanan
9. Order Amount Hike From last Year : right skewed, ada outlier disisi kanan
10. Coupon Used: right skewed, ada outlier disisi kanan
11. Order Count : right skewed, ada outlier disisi kanan
12. cashback amount : right skewed, ada outlier di sisi kiri dan kanan

<b> Untuk data dengan distribusi skew ke kanan, dapat diolah dengan "Feature Transformation", tepatnya dengan standarisasi maupun Log Transformation agar sebaran data menjadi normal. 

In [ ]:
plt.figure(figsize=(17,10))
for i in range(0, len(cats)):
    plt.subplot(3, 3, i+1)
    sns.countplot(x = df_stg1[cats[i]], color='salmon', orient='v')
    plt.xticks(fontsize=9, rotation=45)
    plt.tight_layout()

1. pada feature Prefer Order Cat terdapat 2 kategori yang mirip yaitu, Phone dan Mobile Phone jika kategori tersebut sama, Phone akan direplaace dengan Mobile Phone.
2. Pada Feature Preferred Payment Mode, terdapat variabel yang mirip, yaitu CC dengan Credit Card dan COD dengan Cash On Delivery. CC akan direplace dengan credit card dan COD akan direplae oleh Cash On Delivery.
3. Pada feature PreferedOrderCat terdapat dua kategori yang ambigu, yaitu mobile dan Mobile Phone yang sebenarnya sama. Mobile akan direplace dengan Mobile Phonee. 

## <b> 3. Multivariate Analysis

In [ ]:
df_stg1_1.corr()

In [ ]:
#mask = np.triu(np.ones_like(df_stg1[nums].corr(), dtype=np.bool_))
heatmaps=sns.heatmap(df_stg1_1.corr(),cmap='Blues', annot=True, fmt='.2f')
heatmaps.set_title('Korelasi', fontdict={'fontsize':18}, pad=16)

##### Dari correlation heatmap diatas, ada beberapa feature yang memilki korelasi lemah dengan target kita (Churn), yaitu :

1. number of Device Registration 0.11
2. satisfaction Score 0.11
3. complain 0.25
4. tenure -0.35 (kuat, tetapi korelasi negatif). Feature ini akan menjadi feature penting dalam proses modelling.
5. Days Since Last Order -0.16
6. CashbackAmount -0.15

#####  Dari *correlation heatmap* di atas dapat dilihat bahwa:
* Target kita `churn` memiliki korelasi positif lemah dengan `complain`, `SatisfactionScore`, `NumberOfDeviceRegistered` dan `CityTier` (decent potential feature)
* Sedangkan korelasi `Churn` dengan `HourSpendOnApp`, `NumberOfAddress`, `CouponUsed`, `OrderAmountHikeFromLastyear` dan `OrderCount` sangat lemah ~0, ini menandakan bisa jadi mereka bukan fitur yang potensial.
* `Tenure` memiliki korelasi positif cukup kuat dengan `CashbackAmount`(0.48); `CustomerID` dengan `HourSpendOnApp` (0.6) ; `CouponUsed` dengan `OrderCount`(0.75), `OrderCount` dengan `DaySinceLastOrder` (0.5), Ada kemungkinan *feature* ini redundan (dipilih salah satunya saja untuk training data)

##### Korelasi lainnya 
1. `Churn` berkorelasi negatif dengan `tenure` (korelasi = -0.35), artinya Churn cenderung terjadi pada customer dengan loyalitas yang rendah atau tidak terlalu lama menggunakan layanan. 
2. `Tenure` berkorelasi positif kuat dengan `CashbackAmount`(0.48). Artinya loyalitas dari customer dipengaruhi oleh jumlah cashback yang diberikan oleh E-commerce.  
3. `CashbackAmount` berkorelasi positif dengan `OrderCount`(0.36) dan `DaySinceLastOrder` (0.35). Tentu saja Pemberian dan jumlah cashback sangat mempengaruhi jumlah belanja atau jumlah orderan customer. 
4. `OrderCount` berkorelasi positif kuat dengan `CouponUsed` (0.75). Dimana jumlah orderan dapat dipengaruhi oleh kupon yang tersedia dan dapat digunakan.
5. `CouponUsed` berkorelasi positif dengan `DaySinceLastOrder` (0.36). 

## <b> 4. Business Insight

### 1. Churn Berdasarkan Tenure

In [ ]:
fig, ax = plt.subplots(figsize=(10,7))

sns.kdeplot(df_stg1[df_stg1['Churn']==0]['Tenure'].dropna(),color='navy',label='Churn: No', ax=ax)
sns.kdeplot(df_stg1[df_stg1['Churn']==1]['Tenure'].dropna(),color='orange',label='Churn: Yes', ax=ax)
plt.legend() ; 

In [ ]:
fig, ax = plt.subplots(figsize=(10,7))

sns.histplot(df_stg1[df_stg1['Churn']==0]['Tenure'].dropna(),color='navy',label='Churn: No', ax=ax)
sns.histplot(df_stg1[df_stg1['Churn']==1]['Tenure'].dropna(),color='orange',label='Churn: Yes', ax=ax)
plt.legend()

<b> INSIGHT :
Churn cenderung terjadi pada customer dengan loyalitas yang rendah atau belum terlalu lama menggunakan layanan. kemungkinan customer dengan loyalitas rendah ini adalah customer yang hanya menikmati flash sale atau promo singkat, dll. Perlu dibuat fitur baru dalam layanan e-commerce seperti 'Gaining Poin or Coin' atau game mengumpulkan coin/poin yang nantinya dapat ditukar dengan voucher diskon, gratis ongkir dan lain sebagainya. Sehingga dengan adanya fitur ini, dapat meningkatkan experience dari customer. 

### 2. Kategori Produk yang Paling Banyak Diminati

In [ ]:
df_pref_order = df_stg1.groupby(['PreferedOrderCat','Churn']).agg({'CustomerID':'count'}).reset_index()
df_pref_order['ratio']=df_pref_order['CustomerID'].apply(lambda x:round( x*100.0/(df_pref_order['CustomerID'].sum()),2))
df_pref_order

In [ ]:
sns.barplot(x='PreferedOrderCat', y='CustomerID',hue='Churn', data=df_pref_order);

<b> Kategori laptop & accessory dan mobile Phone menjadi yang paling banyak diburu. Untuk kategori barang yang ingin dijual perlu ditingkatkan lagi atau diseimbangkan lagi. Seperti contohnya untuk kategori fashion, dengan meningkatkan jumlah customer wanita diharapkan penjualan barang dengan kategori fashion ini dapat ditingkatkan. Atau barang yang termasuk dalam kategori others, dengan melakukan marketing campaign seperti flash sale diharapkan mampu menaikkan popularitas barang ini dan meningkatkan penjualannya, sehingga penjualan tidak hanya terfokus pada satu kategori saja.
  

### 3. Churn Berdasarkan Gender

In [ ]:
df_group = df_stg1.groupby(['Gender','Churn']).agg({'CustomerID':'count'}).reset_index()
df_group['ratio']=df_group['CustomerID'].apply(lambda x:round( x*100.0/(df_group['CustomerID'].sum()),2))
df_group

In [ ]:
df_gender = df_stg1.groupby(['Churn', 'Gender']).agg({'CustomerID':'count'}).reset_index()
df_churn = df_stg1.groupby('Churn').agg({'CustomerID':'count'}).reset_index()
df_gender_intotal = df_gender.merge(df_churn, on = 'Churn', how = 'left')
df_gender_intotal['Rasio (%)'] = df_gender_intotal['CustomerID_x']/df_gender_intotal['CustomerID_y']*100
df_gender_intotal

In [ ]:
plt.figure(figsize = (14,10))
sns.barplot(x='Gender', y='Rasio (%)', hue='Churn', data=df_gender_intotal)
plt.legend()

<b> Jumlah Customer pria lebih banyak dibandingkan wanita, Jumlah customer pria yang churn juga lebih banyak dari wanita. Menurut artikel yang dikutip dari Mag for Woman, Wanita lebih suka berbelanja dan merupakan customer yang lebih baik dibandingkan pria. Salah satu alasan wanita menyukai 'shopping' atau berbelanja adalah naluri wanita yang selalu ingin mengumpulkan sesuatu. Pada case ini, jumlah customer wanita lebih sedikit dibandingkan pria, tetapi persentase yang churn justru lebih sedikit dari pria karena wanita memang lebih setia dalam hal berbelanja. Sedangkan pria dalam hal ini kemungkinan hanya mengejar promo dan cashback dari e-commerce ketika berbelanja. Oleh karena itu, perlu dilakukan marketing campaign untuk menarik wanita dan menambah produk-produk yang dirasa dapat menarik wanita untuk berbelanja pada platform kita.
 

### 4. Churn Berdasarkan Complain

In [ ]:
df_group2 = df_stg1.groupby(['Complain','Churn']).agg({'CustomerID':'count'}).reset_index()
df_group_complain = df_stg1.groupby('Complain').agg({'CustomerID':'count'}).reset_index()

In [ ]:
data_complain = df_group2.merge(df_group_complain, on = 'Complain', how = 'left')
data_complain['Rasio (%)'] = data_complain['CustomerID_x']/data_complain['CustomerID_y']*100
data_complain

In [ ]:
conc1= "Persentase Customer  yang Churn dilihat dari Complain"
sns.barplot(x='Complain', y='Rasio (%)', hue='Churn', data=data_complain)
plt.title('Complain X Churn', size = 24)

<b> Customer yang melakukan complain memiliki persentase 31.67% terhadap customer yang churn sehingga dibutuhkan perbaikan dalam penanganan complain (resolve complain). Jika komplain mereka tidak ditangani dengan baik, maka customer yang complain dan belum churn akan berpotensi lebih besar untuk churn pada waktunya karena faktor ketidakpuasan terhadap layanan kita. Solusi yang dapat dilakukan adalah dengan membuat sistem bot untuk mengarahkan customer secara otomatis dan realtime bagaimana cara mengatasi masalah yang memang masih bisa diselesaikan oleh customer sendiri. Kemudian juga membuat fast call service untuk penanganan masalah yang kompleks dengan cepat. 
 

### 5. Tenure vs Cashback Amount

In [ ]:
df_copy = df_stg1.copy()

In [ ]:
klasifikasi = []

for i, kolom in df_copy.iterrows():
    if kolom ['Tenure'] <= 9: 
        result = 'New User  dengan Tenure <=9 bulan'
    elif kolom ['OrderCount'] > 9 and kolom ['OrderCount'] <=20 : 
        result = 'Experienced User dengan Tenure 10-20 bulan'
    else :
        result = 'Loyal User dengan Tenure >20 bulan'
    klasifikasi.append(result)
df_copy['Klasifikasi Berdasarkan Tenure']=klasifikasi
df_copy.head()

In [ ]:
df_tn = df_copy.groupby(['Klasifikasi Berdasarkan Tenure', 'Churn']).agg({'CustomerID' : 'count',
                              'CashbackAmount' : 'sum'})
df_cashback = df_stg1['CashbackAmount'].sum()
df_tn['Rasio(%)'] = df_tn['CashbackAmount']/df_cashback*100
df_tn = df_tn.reset_index()
df_tn = df_tn.rename(columns = {'CustomerID' : 'Jumlah Customer'})
df_tn

In [ ]:
df_tn2 = df_copy.groupby('Klasifikasi Berdasarkan Tenure').agg({'CustomerID' : 'count',
                              'CashbackAmount' : 'sum'})
df_tn2 = df_tn2.reset_index()
df_tn2 = df_tn2.rename(columns = {'CustomerID' : 'Jumlah Customer'})
df_tn2

In [ ]:
sns.barplot(x='Klasifikasi Berdasarkan Tenure', y='CashbackAmount', data=df_tn2, estimator=sum)
plt.title('Tenure vs Cashback Amount', size = 24)
plt.xticks(fontsize=9);

User dengan kategori new user atau dengan lama menggunakan layanan <= 9 bulan adalah customer dengan jumlah paling banyak. sedangkan experienced user dengan tenure 10-20 bulan adalah yang paling sedikit. Jumlah cashback (secara keseluruhan) yang digunakan paling banyak adalah customer dengan kategori loyal user (customer dengan tenure > 20 bulan). 

In [ ]:
sns.barplot(x='Klasifikasi Berdasarkan Tenure', y='CashbackAmount', data=df_tn, estimator=sum, hue='Churn')
plt.title('Tenure vs Cashback Amount', size = 24)
plt.xticks(fontsize=9);

Customer dengan kategori 'new user' atau user dengan lama menggunakan layanan <=9 bulan adalah yang paling banyak churn dan mereka juga yang banyak menggunakan cashback. Hal ini kemungkinan karena customer baru ini hanya ingin menikmati cashback yang disediakan oleh e-commerce. Oleh karena itu, ada baiknya dibuat klasifikasi user berdasarkan lama menggunakan layanan e-commerce kita, agar lebih mudah menargetkan pemberian cashback terhadap customer. Cashback sebaiknya diberikan untuk kategori experience user dan loyal user. Sedangkan untuk new user sebaiknya dibuat strategi lain untuk meningkatkan retensinya selain dengan memberikan cashback, seperti fitur gaining point, games tabur tuai (contoh farming untuk mendapatkan coin/poin) dan lain sebagainya. 

In [ ]:
plt.figure(figsize=(20,15))
sns.barplot(x='CashbackAmount', y='Tenure', data=df_stg1, estimator =sum, orient ='h')
plt.title('Cashback Amount distribution based on Tenure', size = 24);

Jika dilihat dari jumlah `CashbackAmount` yang digunakan, cashback paling banyak diperoleh oleh customer dengan tenure 0-1 bulan. Tindakan ini dilakukan dalam melakukan akusisi pelanggan baru. 

In [ ]:
plt.figure(figsize=(20,15))
sns.barplot(x='CashbackAmount', y='Tenure', data=df_stg1, estimator =sum, hue ='Churn', orient ='h')
plt.title('Cashback Amount distribution based on Tenure', size = 24);

Mayoritas dari customer yang menggunakan cashback adalah customer dengan lama menggunakan layanan kita kurang dari 21 bulan dan customer yang sudah memperoleh cashback lebih cenderung churn. Fenomena ini bisa menjadi pertimbangan dalam pemberian cashback kepada pelanggan. Sungguh disayangkan kita sudah mengalirkan dana operasional berupa cashback kepada pelanggan, dan pelanggan justru churn, karena hanya ingin mengambil cashback selama melakukan pembelian di layanan e-commerce kita. Terlihat jelas bahwa customer yang melakukan order hanya 1 kali, juga paling banyak menggunakan cashback. Kemungkinan memang customer ini hanya ingin mengambil keutungan berupa cashback yang besar sebagai new user. Untuk melakukan akusisi pelanggan baru, juga perlu mempertimbangkan bagaimana cara meningkatkan retensi mereka. Perlu adanya fitur farming/game yang hasilnya dapat ditukarkan dengan syarat, atau pengumpulan poin dari hasil pembelian produk dan lain-lain. Dengan demikian, hal ini dapat meningkatkan retensi dan loyalitas dari new customer.

### 6. Order Count vs Coupon Used

In [ ]:
df_order = df_copy.groupby('OrderCount').agg({'CouponUsed' : 'sum',
                      'CustomerID' : 'count'})
df_order.reset_index()
df_order = df_order.rename(columns = {'CustomerID' : 'Jumlah Customer'})
df_order

In [ ]:
sns.barplot(x='CouponUsed', y='OrderCount', data=df_stg1, estimator=np.mean, hue='Churn', orient ='h')
plt.title('OrderCount vs Rata-rata kupon yang digunakan', size = 24)

Semakin banyak jumlah pesanan/order, semakin banyak kupon yang digunakan oleh customer. Jadi, jumlah order produk itu sangat ditentukan oleh kupon yang kita berikan. Oleh karena itu, kita harus sering memberikan kupon kepada customer dengan mempertimbangkan ROI, BEP dll agar customer tetap setia dan rutin dalam melakukan pemesanan produk. 

### 7. CashbackAmount vs OrderCount

pelanggan juga dapat diklasifikasi dalam beberapa kategor berdasrarkan junlah ordernya. Bronze, Silver dan Gold. 

In [ ]:
klasifikasi = []

for i, kolom in df_copy.iterrows():
    if kolom ['OrderCount'] < 4: 
        result = 'Bronze'
    elif kolom ['OrderCount'] < 9 and kolom ['OrderCount'] >=4 : 
        result = 'Silver'
    else :
        result = 'Gold'
    klasifikasi.append(result)
df_copy['CustomerCategory']=klasifikasi
df_copy.head()

In [ ]:
df_oc = df_copy.groupby(['CustomerCategory', 'Churn']).agg({'CustomerID' : 'count',
                              'CashbackAmount' : 'sum'})
df_cashback = df_copy['CashbackAmount'].sum()
df_oc['Rasio(%)'] = df_oc['CashbackAmount']/df_cashback*100
df_oc.reset_index()
df_oc = df_oc.rename(columns = {'CustomerID' : 'Jumlah Customer'})
df_oc.reset_index()

In [ ]:
df_oc2 = df_copy.groupby('CustomerCategory').agg({'CustomerID' : 'count',
                              'CashbackAmount' : 'sum'})
df_oc2.sort_values(['CustomerID'])
df_oc2 = df_oc2.reset_index()
df_oc2 = df_oc2.rename(columns = {'CustomerID' : 'Jumlah Customer'})
df_oc2

In [ ]:
sns.barplot(x='CustomerCategory', y='Jumlah Customer', data=df_oc2.sort_values(['Jumlah Customer'], ascending = True), estimator=sum)
plt.title('Jumlah Customer berdasarkan Customer Category', size = 24)
plt.xticks(fontsize=9);

Jika dilihat dari klasifikasi customer berdasarkan jumlah order count nya, customer yang memiliki jumlah pesanan kurang dari 4 (bronze) adalah yang paling banyak dan customer dengan jumlah order lebih dari 9 (gold) adalah yang paling sedikit. Kita perlu meningkatkan keinginan customer untuk berbelanja lebih banyak agar meningkatkan revenue perusahaan. Beberapa upaya yang dapat kita lakukan misalnya dengan memberikan cashback dan kupon kepada pelanggan yang memiliki jumlah pesanan dibawah 4 (bronze). 

In [ ]:
sns.barplot(x='Jumlah Customer', y='CashbackAmount', data=df_oc2.sort_values(['CustomerCategory'], ascending = True), estimator=sum)
plt.title('CashbackAmount vs OrderCount', size = 24)
plt.xticks(fontsize=9);

Jumlah cashback paling banyak digunakan oleh customer kategori Bronze

In [ ]:
sns.barplot(x='CustomerCategory', y='CashbackAmount', data=df_copy.sort_values(['CustomerID'], ascending = True), estimator=sum, hue='Churn')
plt.title('CashbackAmount vs OrderCount', size = 24)
plt.xticks(fontsize=9);

Pengguna Bronze yang menggunakan Cashback paling banyak Churn dibandingkan pengguna silver dan Gold. 

In [ ]:
sns.barplot(x='CashbackAmount', y='OrderCount', data=df_stg1,  estimator=sum, hue='Churn', orient ='h')
plt.title('CashbackAmount vs OrderCount', size = 24)

Jika dilihat berdasarkan jumlah order nya, pengguna dengan order yang sedikit, atau dapat dikatakan order count < 4 cenderung melakukan churn dibanding customer yang sudah melakukan banyak order/pesanan/pembelian produk. Customer yang memiliki ordercount 1-2 cenderung churn, karena pada masa ini adalah masa akusisi pelanggan baru. Perlu dilakukan strategi yang telah disebutkan di atas untuk meningkatkan retensi pelanggan dan experience dari user terhadap layanan e commerce kita. 

### 8. Tenure vs Satisfaction Score

In [ ]:
plt.figure(figsize = (20,10))
sns.barplot(x='SatisfactionScore', y='Tenure', data=df_stg1, hue='Churn', orient ='h')
plt.title('Tenure vs SatisfactionScore', size = 24)

In [ ]:
df_satisfaction = df_copy.groupby(['Tenure', 'SatisfactionScore']).agg({'CustomerID' : 'count'})
df_satisfaction.reset_index()

In [ ]:
plt.figure(figsize = (20,10))
df_satisfaction.plot()
plt.title('Tenure vs SatisfactionScore', size = 24)

# Stage 2 - DATA PRE-PROCESSING

In [ ]:
df.head()

## <B> 1. Data Cleansing

<b> GENERAL INFORMATION

In [ ]:
# informasi general tentang dataframe
df.info()

### Statistical Summary

In [ ]:
# pengelompokan kolom berdasarkan jenisnya
nums = ['Tenure','CityTier', 'WarehouseToHome', 'HourSpendOnApp', 'NumberOfDeviceRegistered','SatisfactionScore', 'DaySinceLastOrder','NumberOfAddress', 'OrderAmountHikeFromlastYear', 'CouponUsed', 'OrderCount', 'CashbackAmount']
cats = ['PreferredLoginDevice', 'PreferredPaymentMode', 'Gender', 'PreferedOrderCat', 'MaritalStatus']

In [ ]:
df[nums].describe().transpose()

Beberapa kolom/feature sudah cukup simetrik distribusinya (mean dan median tidak terlalu jauh), namun ada beberapa kolom yang  skew, dilihat dari nilai mean dan median ---->        "mean>median" = skew ke kanan (akan kita lihat visualisasi nya di bawah) yaitu kolom `Tenure`, `WareHouseToHome`, `OrderCount`,`DaySinceLastOrder`,`CashbackAmount`, `NumberOfAddress`, `OrderAmountHikeFromlastYear`  dst . kolom-kolom ini mungkin dapat dilakukan log transformation nantinya. 

In [ ]:
df[cats].describe().transpose()

1. Customer di dominasi oleh pria. 
2. customer mayoritas lebih memilih login melalui mobile phone
3. metode pembayaran yang digunakan kebanyakan dengan metode debit card
4. kategori produk yang diorder mayoritas adalah laptop dan aksesoris
5. Mayoritas customer berstatus menikah.

### A. Handling Missing Values

Kita cek terlebih dahulu jumlah missing value pada dataset kita, dalam jumlah dan dalam persen

In [ ]:
# jumlah entry NULL di setiap kolom
df.isna().sum()

In [ ]:
percentage_of_null_values = ((df.isnull().sum())*100/len(df)).sort_values(ascending = False)
percentage_of_null_values

<b> Note :
* Untuk kolom/feature yang memiliki missing value, karena nilai null > 5% data, maka akan dilakukan imputation berdasarkan sebaran/distribusi datanya. jika data skew ke kanan (mean>median), maka digunakan nilai median karena nilai median robust terhadap outlier. jika mean< median, digunakan nilai mean. 
* ada opsi lain untuk proses imputation ini, yaitu menggunakan nilai IQR (Q3-Q1) yang juga robust terhadap outlier. dalam case ini, kami hanya menggunakan nilai median dan mean saja. 

Berikut adalah strategi kita mengatasi missing values pada setiap kolom. 
<br>
INGAT: jikalau kita melakukan imputasi, maka itu mencerminkan ASUMSI kita.
* `DaySinceLastOrder` \: impute dengan nilai median (dari `df.describe` kita tau distribusinya mendekati skewed ke kanan)
* `OrderAmountHikeFromlastYear` \: impute dengan nilai median (dari `df.describe` kita tau distribusinya mendekati skewed ke kanan)
* `Tenure ` \: impute dengan nilai median (dari `df.describe` kita tau distribusinya mendekati skewed ke kanan)
* `OrderCount` \: impute dengan nilai median (dari `df.describe` kita tau distribusinya mendekati skewed ke kanan)
* `CouponUsed`\: impute dengan nilai median (dari `df.describe` kita tau distribusinya mendekati skewed ke kanan)
* `HourSpendOnApp`\: impute dengan nilai average (dari `df.describe` kita tau distribusinya mendekati normal)
* `WarehouseToHome`\: impute dengan nilai median (dari `df.describe` kita tau distribusinya mendekati skewed ke kanan)

##### PROSES IMPUTASI

In [ ]:
# Impute DaySinceLastOrder, OrderAmountHikeFromlastYear, Tenure, OrderCount, CouponUsed, dan WarehouseToHome dengan median
df['DaySinceLastOrder'].fillna(df['DaySinceLastOrder'].median(), inplace=True)
df['OrderAmountHikeFromlastYear'].fillna(df['OrderAmountHikeFromlastYear'].median(), inplace=True)
df['Tenure'].fillna(df['Tenure'].median(), inplace=True)
df['OrderCount'].fillna(df['OrderCount'].median(), inplace=True)
df['CouponUsed'].fillna(df['CouponUsed'].median(), inplace=True)
df['WarehouseToHome'].fillna(df['WarehouseToHome'].median(), inplace=True)

In [ ]:
#missing value kolom `HourSpendOnApp` diisi dengan mean, karena pada deskriptif statistiknya mean<median. 
df['HourSpendOnApp'].fillna(df['HourSpendOnApp'].mean(), inplace=True)

<b> Dicek kembali missing value setelah proses imputation

In [ ]:
# cek jumlah missing data setelah preprocessing
df.isna().sum()

Setelah dilakukan imputation, statistik deskriptif berubah, kita cek ulang

In [ ]:
df[nums].describe().transpose()

deskriptif statistik setelah proses imputasi tidak berubah secara signifikan. data setelah imputasi relatif masih mendekati nilai originalnya sebelum imputasi. 

### B. Duplicated Data

In [ ]:
# cek jumlah duplicated rows
# dari semua kolom
print('Jumlah data yang duplikat adalah :', df.duplicated().sum())

In [ ]:
# cek jumlah duplicated rows --> COBA-COBA
# dari 3 kolom saja (CustomerID, Churn, Complain)
df.duplicated(subset=['CustomerID', 'Churn', 'Complain']).sum()

<b> Karena tidak ada data yang duplikat, maka selanjutnya dilakukan handling outliers

### C. Outliers Handling

Buat copy dataset yang sudah dilakukan Handling Missing Value dan Handling Duplicated Data

In [ ]:
df.head()

In [ ]:
df_a = df.copy()

#### 1. Menggunakan Z Score 

Kolom numerical yang dapat menggunakan Z-score untuk Outliers Handling adalah `HourSpendOnApp` dan `NumberOfDeviceRegistered`
(Dapat dilihat pada boxplot, dua kolom ini memiliki distribusi normal dan memiliki outlier). Sedangkan kolom `SatisfactionScore` tidak perlu lagi dilakukan Outliers Handling karena kolom ini tidak memiliki outliers. 

In [ ]:
from scipy import stats

print(f'Jumlah baris sebelum memfilter outlier: {len(df_a)}')

filtered_entries = np.array([True] * len(df_a))

for col in df[nums]:
    zscore = abs(stats.zscore(df_a[col])) # hitung absolute z-scorenya
    filtered_entries = (zscore < 3) & filtered_entries # keep yang kurang dari 3 absolute z-scorenya
    
df_z = df_a[filtered_entries] # filter, cuma ambil yang z-scorenya dibawah 3

print(f'Jumlah baris setelah memfilter outlier: {len(df_z)}')
print('Maka outlier yang dihapus sekitar', round((len(df_a)-len(df_z))/len(df_a)*100), '%')

Jika kita menggunakan metode IQR, kemungkinan besar data yang hilang sangat banyak  (sangat ekstrim). Padahal jika dilihat dari boxplot, hampir semua feature yang memiliki outlier.  Dengan demikian, direkomendasikan melakukan z-score saja untuk handling outliers. selanjutnya dilakukan feature transformation untuk mengubah range value nya (tindakan ini juga mencegah adanya outlier yang terlalu ekstrim yang dapat mengacaukan pemodelan nantinya). kita ambil contoh IQR dibawah ini :

In [ ]:
total_customer = df_z.shape[0]
a = df_z.groupby(['Churn'])['CustomerID'].count().reset_index()
a['Persentase(%)'] = a['CustomerID']/total_customer*100
a

In [ ]:
import matplotlib.pyplot as plt
 
A=['Tidak Churn', 'Churn']
T=a['Persentase(%)']
  
plt.pie(T, labels=A,autopct= '%1.1f%%')
plt.title('Churn Proportion', size = 26)
plt.show()

#### <b> Contoh handling outlier dengan IQR

In [ ]:
# Untuk kolom HourSpendOnApp
df_b = df.copy()
Q1_s = df_b['HourSpendOnApp'].quantile(0.25)
Q3_s = df_b['HourSpendOnApp'].quantile(0.75)
IQR_s = Q3_s-Q1_s
low_limit = Q1_s -(1.5*IQR_s)
high_limit = Q3_s + (1.5*IQR_s)
filtered_entries = ((df_b['HourSpendOnApp'] >= low_limit) & (df_b['HourSpendOnApp'] < high_limit))
df_x = df_b[filtered_entries]
print('Jumlah baris sebelum memfilter outlier :', len(df_b))
print('Jumlah baris setelah memfilter outlier :', len(df_x))

In [ ]:
# Untuk kolom NumberOfDeviceRegistered
df_c = df.copy()
Q1_n = df_c['NumberOfDeviceRegistered'].quantile(0.25)
Q3_n = df_c['NumberOfDeviceRegistered'].quantile(0.75)
IQR_n = Q3_n-Q1_n
low_limit = Q1_n -(1.5*IQR_n)
high_limit = Q3_n + (1.5*IQR_n)
filtered_entries = ((df_c['NumberOfDeviceRegistered'] >= low_limit) & (df_c['NumberOfDeviceRegistered'] < high_limit))
df_y = df_c[filtered_entries]
print('Jumlah baris sebelum memfilter outlier :', len(df_c))
print('Jumlah baris setelah memfilter outlier :', len(df_y))

In [ ]:
# jika disesuaikan kembali data yang dihapus dari kedua feature ini, maka 
df_oke = df_x.merge(df_y, on = 'CustomerID')
print('Jumlah baris setelah memfilter outlier dengan IQR dari kolom HourSpendOnApp dan NumberOfDeviceRegistered :', len(df_oke))
print('Maka outlier yang dihapus sekitar', ((len(df_c)-len(df_oke))/len(df_c)*100), '%')

data di atas masih handling ourlier dari 2 feature. bagaiman jika 10? pasti outlier yang dihilangkan akan lebih banyak sehingga mengganggu keseimbangan data, maupun pemodelan. dari trial di atas, metode IQR terlalu ekstrim dalam metode outlier handling pada case ini. 

-------------

#### Mengubah data yang tidak konsisten---> satu definisi, dua entry. 
contoh : phone dan mobile phone. 

In [ ]:
df_z['PreferredLoginDevice'].replace('Phone', 'Mobile Phone', inplace = True)
df_z['PreferredPaymentMode'].replace('CC', 'Credit Card', inplace = True)
df_z['PreferredPaymentMode'].replace('COD', 'Cash on Delivery', inplace = True)
df_z['PreferedOrderCat'].replace('Mobile', 'Mobile Phone', inplace = True)

Cek kembali setelah di-replace

In [ ]:
for col in df_z[cats]:
  print(f'value counts of column {col}')
  print(df_z[col].value_counts())
  print('---'*10, '\n')


### BUSSINESS INSIGHT

#### 1. Kategori Produk yang paling banyak diminati

In [ ]:
df_prefered_order = df_z.groupby(['PreferedOrderCat']).agg({'CustomerID':'count'}).reset_index()
df_prefered_order['ratio']=df_prefered_order['CustomerID'].apply(lambda x:round( x*100.0/(df_prefered_order['CustomerID'].sum()),2))
df_prefered_order

In [ ]:
sns.barplot(x='PreferedOrderCat', y='CustomerID', data=df_prefered_order);

Kategori mobile Phone dan laptop & accesory menjadi yang paling banyak diburu

In [ ]:
cats_update1 = ['MaritalStatus', 'PreferredLoginDevice', 'PreferredPaymentMode', 'Gender', 'PreferedOrderCat']
df_z[cats].describe().transpose()

#### 2. Churn by Tenure

In [ ]:
fig, ax = plt.subplots(figsize=(10,7))

sns.kdeplot(df_z[df_z['Churn']==0]['Tenure'].dropna(),color='navy',label='Churn: No', ax=ax)
sns.kdeplot(df_z[df_z['Churn']==1]['Tenure'].dropna(),color='orange',label='Churn: Yes', ax=ax)
plt.legend() ; 

In [ ]:
fig, ax = plt.subplots(figsize=(10,7))

sns.histplot(df_z[df_z['Churn']==0]['Tenure'].dropna(),color='navy',label='Churn: No', ax=ax)
sns.histplot(df_z[df_z['Churn']==1]['Tenure'].dropna(),color='orange',label='Churn: Yes', ax=ax)
plt.legend()

<b> INSIGHT :
Churn cenderung terjadi pada customer dengan loyalitas yang rendah atau belum terlalu lama menggunakan layanan. kemungkinan customer dengan loyalitas rendah ini adalah customer yang hanya menikmati flash sale atau promo singkat, dll. Pada masa 0-1 bulan adalah masa pelanggan baru menggunakan layanan e-commerce kita. Perlu dibuat fitur baru dalam layanan e-commerce seperti 'Gaining Poin or Coin' atau game mengumpulkan coin/poin tersebut yang nantinya dapat ditukar untuk meningkatkan rasa penasaran, experience dan ambisi dari customer sehingga dapat meningkatkan retensi dari pelanggan.

#### 3. Churn by Complain

In [ ]:
df_complain = df_z.groupby(['Complain','Churn']).agg({'CustomerID':'count'}).reset_index()
df_complained = df_z.groupby('Complain').agg({'CustomerID':'count'}).reset_index()

In [ ]:
data_complains = df_complain.merge(df_complained, on = 'Complain', how = 'left')
data_complains['Rasio (%)'] = data_complains['CustomerID_x']/data_complains['CustomerID_y']*100
data_complains

In [ ]:
conc1= "Persentase Customer  yang Churn dilihat dari Complain"
sns.barplot(x='Complain', y='Rasio (%)', hue='Churn', data=data_complains)
plt.title('Complain X Churn', size = 24)

Customer yang melakukan complain memiliki persentase 31.67% terhadap customer yang churn sehingga dibutuhkan perbaikan dalam penanganan complain (resolve complain). Jika komplain mereka tidak ditangani dengan baik, maka potensi customer yang complain dan belum churn akan churn pada waktunya karena faktor ketidakpuasan terhadap layanan kita.

#### 4. CHURN VS MARITAL STATUS

In [ ]:
df_status = df_z.groupby(['MaritalStatus','Churn']).agg({'CustomerID':'count'}).reset_index()
df_statuss = df_z.groupby('MaritalStatus').agg({'CustomerID':'count'}).reset_index()

In [ ]:
data_marital_status = df_status.merge(df_statuss, on = 'MaritalStatus', how = 'left')
data_marital_status['Rasio (%)'] = data_marital_status['CustomerID_x']/data_marital_status['CustomerID_y']*100
data_marital_status

In [ ]:
conc1= "Persentase Customer  yang Churn dilihat dari Status Pernikahannya"
sns.barplot(x='MaritalStatus', y='Rasio (%)', hue='Churn', data=data_marital_status)
plt.title('Marital Status X Churn', size = 24)

Customer yang single lebih cenderung churn. Seseorang yang single memang memiliki kebiasaan untuk memilih. Memilih yang lebih banyak cashback nya (kemungkinan punya kompetitor lehih banyak cashbacknya), coupon nya dan keuntungan lainnya. Dibandingkan customer yang sudah menikah ataupun yang sudah pernah menikah, mereke cenderung lebih setiap atau bergantung kepada 1 platform e commerce yang selalu mereka gunakan, karena mereka sudah riset terlebih dahulu mengenai realibilitas, kemudahan dan keuntungan selama menggunakan layanan e commerce tersebut. selain itu, mereka juga sudah mulai terbiasa menjadi perilaku konsumtif dengan memanfaatkan layanan e commerce untuk melengkapi hampir seluruh kebutuhan mereka (sekunder).  

### D. Feature transformation

#### 1. NORMALIZATION/STANDARIZATION

In [ ]:
df_z.describe().transpose()

-------------

* <b>  `Tenure` dan `WarehouseToHome`, distandarisasi MERUBAH bentuk sebaran data menjadi mendekati distribusi normal/mendekati, maka di lakukan standarisasi 
* <b> `CashbackAmount` , `NumberOfDeviceRegistered` dan `HourSpendOnApp` di normalisasi

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# CouponUsed dan OrderCount kita re-scale ke [0,1] (normalisasi)
df_z['CashbackAmount_normalisasi'] = MinMaxScaler().fit_transform(df_z['CashbackAmount'].values.reshape(len(df_z), 1))
df_z['NumberOfDeviceRegistered_normalisasi'] = MinMaxScaler().fit_transform(df_z['NumberOfDeviceRegistered'].values.reshape(len(df_z), 1))
df_z['DaySinceLastOrder_normalisasi'] = MinMaxScaler().fit_transform(df_z['DaySinceLastOrder'].values.reshape(len(df_z), 1))

# Tenure dan Warehouse perlu distandarisasi untuk mendapatkan data dengan distribusi normal
df_z['std_Tenure'] = StandardScaler().fit_transform(df_z['Tenure'].values.reshape(len(df_z), 1))
df_z['std_WarehouseToHome'] = StandardScaler().fit_transform(df_z['WarehouseToHome'].values.reshape(len(df_z), 1))

# OrderAmountHikeFromlastYear tidak perlu ditransformasi
# HourSpendOnApp tidak perlu di transformasi
# CouponUsed tidak perlu di transformasi
# OrderCount tidak perlu ditransformasi
# NumberofAddress tidak perlu ditransformasi
df_z

<b> Feature `Standarisasi` digunakan dalam feature transformation karena :
* Feature ini dapat merubah bentuk sebaran menjadi mendekati distribusi normal
* Range ketika standarisasi tidak terlalu loose dan nilainya adalah real. tidak seperti log transformation yang memungkinkan nilainya NaN dan Infinity (karena value dari feature). jadi, log transformation tidak disarankan pada case ini. 

In [ ]:
# cek keberhasilan feature transformation
df_z.describe().transpose()

In [ ]:
nums_after_transform = ['std_Tenure', 'std_WarehouseToHome', 'HourSpendOnApp', 'NumberOfDeviceRegistered_normalisasi','SatisfactionScore', 'DaySinceLastOrder_normalisasi','NumberOfAddress', 'OrderAmountHikeFromlastYear', 'CouponUsed', 'OrderCount', 'CashbackAmount_normalisasi']

In [ ]:
plt.figure(figsize=(30,20))
features = nums_after_transform
for i in range(0, len(features)):
    plt.subplot(3, 7, i+1)
    sns.boxplot(y=df_z[features[i]], orient='v')
    plt.xlabel(features[i])

<b> Note :
* Jika dilihat dari distribusi data setelah standarisasi, range/point antar value tidak terlalu ekstrim, atau masih dapat ditolerir maka data ini sudah aman untuk dilakukan proses selanjutnya

In [ ]:
features = nums_after_transform
plt.figure(figsize=(25, 15))
for i in range(0, len(features)):
    plt.subplot(3, 5, i+1)
    sns.kdeplot(x=df_z[features[i]], color='salmon')
    plt.xlabel(features[i])

### E. FEATURE ENCODING

In [ ]:
df_z.head()

In [ ]:
df_z.shape

In [ ]:
cats_update1 = ['MaritalStatus', 'PreferredLoginDevice', 'PreferredPaymentMode', 'Gender', 'PreferedOrderCat']
df_z[cats].describe().transpose()

----------------

### E. FEATURE ENCODING

In [ ]:
cats_updated = ['PreferredLoginDevice', 'PreferredPaymentMode', 'PreferedOrderCat', 'MaritalStatus']

<b>Strategi encoding
* `Gender` \: label encoding
* `MaritalStatus`, `PreferredLoginDevice`, `PreferedOrderCat` \: One Hot Encoding 

In [ ]:
# Mapping Gender
mapping_gender = {
    'Female' : 0,
    'Male' : 1
}
df_z['Gender'] = df_z['Gender'].map(mapping_gender)

In [ ]:
df_z['Gender'] = df_z['Gender'].map(mapping_gender)
for cat in ['MaritalStatus', 'PreferredLoginDevice', 'PreferedOrderCat']:
    onehots = pd.get_dummies(df_z[cat], prefix=cat)
    df_z = df_z.join(onehots)
#PreferedPaymentMethod tidak perlu di feature encoding. pengaruh feature ini terhadap label sangat kecil

In [ ]:
df_z.info()

### F. HANDLE CLASS IMBALANCE 

In [ ]:
# pembuatan binary label target yang imbalance (sebagai contoh)
df_z['churn_class'] = df_z['Churn'] > 0.8
df_z['churn_class'].value_counts()

In [ ]:
# pemisahan features vs target
X = df_z[[col for col in df_z.columns if (str(df_z[col].dtype) != 'object') and col not in ['Churn', 'churn_class']]]
y = df_z['churn_class'].values
print(X.shape)
print(y.shape)

HANDLING IMBALANCE ADA PADA SAAT PEMODELAN

<B> PADA STAGE 3 NANTI,  DATA TRAINING YANG DI SMOTE, 
DATA TEST JANGAN DILAKUKAN DATA IMBALANCING

## 2. Feature Engineering

### A. Feature selection (membuang feature yang kurang relevan atau redundan)

#### Drop kolom-kolom yang outdated

In [ ]:
df_z1 = df_z.copy()
df_z1.head()

In [ ]:
# drop kolom2 yang asli (karena sudah di-encoding)
df_z1= df_z1.drop(columns=['PreferredLoginDevice', 'MaritalStatus','PreferredPaymentMode','PreferedOrderCat'])

In [ ]:
#melihat kembali korelasi fitur terhadap label (churn)
plt.figure(figsize = (30, 26))
heatmaps=sns.heatmap(df_z1.corr(),cmap='Blues', annot=True, fmt='.2f')
heatmaps.set_title('Korelasi', fontdict={'fontsize':18}, pad=16)

In [ ]:
# drop kolom2 berikut (ada yang sudah dilakukan Feature Transformation, ada yang tidak perlu)
df_z2= df_z1.drop(columns=['CustomerID', 'Tenure','NumberOfDeviceRegistered', 'WarehouseToHome',
                        'DaySinceLastOrder', 'CashbackAmount', 'OrderAmountHikeFromlastYear', 'CouponUsed', 'Gender' ])
# Tenure telah ditransformasi
# OrderAmountHikeFromlastYear memiliki korelasi yang sangat kecil terhadap label
# CouponUsed memiliki korelasi yang sangat kecil terhadap label
# OrderCount memiliki korelasi yang sangat kecil terhadap label
# CashbackAmount telah di normalisasi
# DaySinceLastOrder telah di normalisasi
# NumberOfDeviceRegistered telah di transformasi


In [ ]:
#melihat kembali korelasi fitur terhadap label (churn)
plt.figure(figsize = (30, 26))
heatmaps=sns.heatmap(df_z2.corr(),cmap='Blues', annot=True, fmt='.2f')
heatmaps.set_title('Korelasi', fontdict={'fontsize':18}, pad=16)

<b> setelah menghapus beberapa fitur yang tidak terlalu potensial :

In [ ]:
df_z2.info()

### B. Feature extraction (membuat feature baru dari feature yang sudah ada)

In [ ]:
df_z2.head()

In [ ]:
df_z2.describe().transpose()

Dibuat segmentasi customer berdasarkan order count. Jika jumlah ordernya 2-3, dikategorikan sebagai Bronze, Jika order nya sebanyak 4-8 dikategorikan sebagai silver dan lebih dari 8 dikategorikan sebagai Gold. 

In [ ]:
klasifikasi = []

for i, kolom in df_z2.iterrows():
    if kolom ['OrderCount'] < 4: 
        result = 'Bronze'
    elif kolom ['OrderCount'] < 9 and kolom ['OrderCount'] >=8 : 
        result = 'Silver'
    else :
        result = 'Gold'
    klasifikasi.append(result)
df_z2['CustomerCategory']=klasifikasi
df_z2

In [ ]:
# Mapping CustomerCategory
mapping_CustomerCategory = {
    'Bronze' : 0,
    'Silver' : 1,
    'Gold' : 2
}
df_z2['CustomerCategory'] = df_z2['CustomerCategory'].map(mapping_CustomerCategory)

In [ ]:
df_z2.info()

In [ ]:
# drop kolom2 ordercount
df_z2= df_z2.drop(columns=['OrderCount'])

In [ ]:
df_z2.info()

In [ ]:
#melihat kembali korelasi fitur terhadap label (churn)
plt.figure(figsize = (30, 26))
heatmaps=sns.heatmap(df_z2.corr(),cmap='Blues', annot=True, fmt='.2f')
heatmaps.set_title('Korelasi', fontdict={'fontsize':18}, pad=16)

### C. Tuliskan minimal 4 feature tambahan

* <b> Membuat suka-tidak suka dari satisfaction score,
sangat tidak puas - tidak puas - cukup puas - puas -sangat puas
* <b> Penghasilan → dapat digunakan untuk melihat apakah besar pernghasilan yang dimiliki seorang customer berpengaruh terhadap kemampuan beli customer tersebut atau tidak.
* <b> Umur → Data umur customer dapat digunakan dalam pengelompokan customer dan melihat kecenderungan ketertarikan produk dalam setiap kelompok, yang kemudian dapat digunakan untuk melakukan promosi terhadap produk-produk yang mungkin disukai oleh kelompok usia tertentu.
* <b> Total Charges → Dengan mengetahui jumlah pengeluaran seorang customer di e-commerce, dapat diprediksi seberapa loyal customer tersebut.
* <b> Total transaction/spend → Informasi ini bermanfaat dalam memprediksi seberapa sering seorang customer melakukan transaksi.
* <b> Profession → Data mengenai profesi yang dimiliki seorang customer dapat digunakan untuk membuat pengklasifikasian, profesi apa lebih banyak melakukan pembelian kategori produk apa.
* <b> Ongkos kirim → Besar biaya pengiriman yang dibebankan kepada customer, juga dapat digunakan untuk memprediksi seberapa loyal seorang customer.
* <b> Transaction Time/waktu transaksi → Informasi mengenai waktu transaksi ini dapat digunakan dalam memperkirakan waktu yang tepat ketika hendak melakukan promosi, sehingga promosi yang dilakukan dapat menjangkau lebih banyak customer. <br><br>

Dengan tambahan feature-feature di atas, kami berpendapat bahwa model machine learning yang dibuat akan semakin efektif dan tepat sasaran.

# Stage 3 - MODELLING

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, auc

def eval_classification(model, pred, xtrain, ytrain, xtest, ytest):
    print("Accuracy (Test Set): %.2f" % accuracy_score(ytest, pred))
    print("Precision (Test Set): %.2f" % precision_score(ytest, pred))
    print("Recall (Test Set): %.2f" % recall_score(ytest, pred))
    print("F1-Score (Test Set): %.2f" % f1_score(ytest, pred))
    
    fpr, tpr, thresholds = roc_curve(ytest, pred, pos_label=1) # pos_label: label yang kita anggap positive
    print("AUC: %.2f" % auc(fpr, tpr))

def show_feature_importance(model):
    feat_importances = pd.Series(model.feature_importances_, index=X.columns)
    ax = feat_importances.nlargest(25).plot(kind='barh', figsize=(10, 8))
    ax.invert_yaxis()

    plt.xlabel('score')
    plt.ylabel('feature')
    plt.title('feature importance score')

def show_best_hyperparameter(model, hyperparameters):
    for key, value in hyperparameters.items() :
        print('Best '+key+':', model.get_params()[key])

## CLASSIFICATION

In [ ]:
df_model = df_z2.copy()

In [ ]:
# pemisahan features vs target
X = df_model[[col for col in df_model.columns if (str(df_model[col].dtype) != 'object') and col not in ['Churn', 'churn_class']]]
y = df_model['churn_class'].values
print(X.shape)
print(y.shape)

In [ ]:
df_model.info()

In [ ]:
#Splitting the data into Train & Test
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, random_state = 42)

In [ ]:
from imblearn import under_sampling, over_sampling
X_under, y_under = under_sampling.RandomUnderSampler(1).fit_resample(X_train, y_train)
X_over, y_over = over_sampling.RandomOverSampler(1).fit_resample(X_train, y_train)
X_over_SMOTE, y_over_SMOTE = over_sampling.SMOTE(1).fit_resample(X_train, y_train)

In [ ]:
print('Original')
print(pd.Series(y).value_counts())
print('\n')
print('UNDERSAMPLING')
print(pd.Series(y_under).value_counts())
print('\n')
print('OVERSAMPLING')
print(pd.Series(y_over).value_counts())
print('\n')
print('SMOTE')
print(pd.Series(y_over_SMOTE).value_counts())

## A. Logistic Regression


In [ ]:
#inisialisasi object logistic regression
model = LogisticRegression(random_state=42)

#fitting model logistic regression
model.fit(X_over_SMOTE, y_over_SMOTE)

<b>Prediction Result (in data test)</b>

In [ ]:
y_pred = model.predict(X_test)
y_pred

In [ ]:
from sklearn.metrics import recall_score
recall_score(y_test, y_pred)

In [ ]:
y_pred_proba = model.predict_proba(X_test)
y_pred_proba

<b>Evaluation</b>

In [ ]:
eval_classification(model, y_pred, X_over_SMOTE, y_over_SMOTE, X_test, y_test)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_pred, y_test)

- True positive = 1141
- False Negative = 44
- False Positive = 198
- True Negative = 222

Case :
- True positive : Kita memprediksi bahwa customer akan churn, dan prediksinya benar
- False Positive : Kita memprediksi bahwa customer akan churn, akan tetapi faktanya dia tidak churn. 
- False Negative : Kita memprediksi bahwa customer tidak churn, prediksi tidak tepat dan aktualnya customer -> churn.
- True Negative : Kita memprediksi bahwa customer tidak churn, dan prediksi kita benar

Matrix :
- Accuracy : Digunakan ketika label sama/memiliki kepentingan yang sama. 
- Precision : Digunakan ketika kita ingin lebih memperhatikan jumlah False Positive (FP) yang sebaiknya lebih sedikit. 
- Recall : Kita menginginkan nilai atau jumlah dari False Negative sekecil mungkin. karena akan berakibat pada penurunan revenue atau pendapatan perusahaan. 
- F1-Score :
- AUC :


Berdasarkan matrix yang disebutkan di atas, kita akan fokus pada matrix Recall, karena :
- Jika False Negative besar, maka akan berpengaruh langsung pada penurunan revenue perusahaan
- Jika False negative besar, maka upaya untuk mencari customer baru/akusisi customer akan memakan operational cost yang lebih besar dibandingkan mempertahankan pelanggan lama. 


Kita juga fokus pada matrix Precision, karena :
- False Positive besar, yang artinya kita salah memprediksi customer yang seharusnya tidak churn, model kita memprediksi churn. Jika False Posotive besar, maka secara teknis kita telah salah sasaran dalam menempatkan operational cost dalam retensi pelanggan, yang mana cost tersebut seharusnya dapat dipakai untuk operasional lainnya seperti marketting campaign dll. 

<b> Model Logistic Regresion telah memberikan score yang cukup baik untuk metric recall, akan tetapi masih kurang baik untuk metric Precision. Akan dilakukan Tuning Hyperparameter untuk meningkatkan model, sehingga kinerja model dalam prediksi lebih baik. 

###  Randomized Search

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

# List Hyperparameters yang akan diuji
penalty = ['l2','l1','elasticnet']
#l1 --> Lasso
#l2 --> ridge
C = [0.0001, 0.001, 0.002] # Inverse of regularization strength; smaller values specify stronger regularization.
hyperparameters = dict(penalty=penalty, C=C)

# Inisiasi model
logres = LogisticRegression(random_state=42) # Init Logres dengan Gridsearch, cross validation = 5
model = RandomizedSearchCV(logres, hyperparameters, cv=5, random_state=42, scoring='recall')

# Fitting Model & Evaluation
model.fit(X_over_SMOTE, y_over_SMOTE)
y_pred = model.predict(X_test)
eval_classification(model, y_pred, X_over_SMOTE, y_over_SMOTE, X_test, y_test)

In [ ]:
print('Best algorithm:', model.best_estimator_.get_params()['penalty'])
print('Best C:', model.best_estimator_.get_params()['C'])

In [ ]:
print('Train score: ' + str(model.score(X_train, y_train))) #recall
print('Test score:' + str(model.score(X_test, y_test))) #recall

<b> setelah dilakukan hyperparameter tuning, score yang diperoleh juga belum cukup baik pada metric precision. Jika dilihat dari score recallnya, model yang diperoleh masih Overfit (gap antara test score dan train score sangat besar). Next : Lakukan Trial lagi untuk model lain. 

## B. K-nearest Neighbor


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_over_SMOTE, y_over_SMOTE)

# Prediction & Evaluation
y_pred = model.predict(X_test)
eval_classification(knn, y_pred, X_over_SMOTE, X_over_SMOTE, X_test, y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_pred, y_test)

- True positive =1007
- False Negative = 32
- False Positive = 332
- True Negative = 234

<b> Nilai False negative sudah kecil, akan tetapi False Positive sangat tinggi. perlu dilakukan percobaan hyperparameter tuning. Model ini belum cukup baik untuk bisnis. Karena jumlah False Positive ini masih sangat berpengaruh terhadap operasional cost. jika jumlahnya terlalu besar seperti di atas, maka akan terjadi penempatan cost yang salah. Perlu dilakukan tuning hyperparameter ataupun  model lain. 

### Tune Hyperparameter with RandomSearch
Setiap algoritma mempunyai list of hyperparameter yang berbeda-beda. lihat list hyperparameternya di dokumentasi sklearn.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

# List of hyperparameter
n_neighbors = list(range(1,30))
p=[1,2]
algorithm = ['auto', 'ball_tree', 'kd_tree', 'brute']
hyperparameters = dict(n_neighbors=n_neighbors, p=p, algorithm=algorithm)

# Init model
knn = KNeighborsClassifier()
model = RandomizedSearchCV(knn, hyperparameters, cv=5, random_state=42, scoring='recall')

# Fit Model & Evaluasi
model.fit(X_over_SMOTE, y_over_SMOTE)
y_pred = model.predict(X_test)
eval_classification(model, y_pred, X_over_SMOTE, y_over_SMOTE, X_test, y_test)

In [ ]:
print('Best n_neighbors:', model.best_estimator_.get_params()['n_neighbors'])
print('Best p:', model.best_estimator_.get_params()['p'])
print('Best algorithm:', model.best_estimator_.get_params()['algorithm'])

In [ ]:
print('Train score: ' + str(model.score(X_over_SMOTE, y_over_SMOTE))) #recall
print('Test score:' + str(model.score(X_test, y_test))) #recall

<b> Masih Overfit. gap antara Train score dan test score masih terlalu besar. perlu optimasi dengan model lain. 

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_pred, y_test)

- True positive = 1168
- False Negative = 43
- False Positive = 171
- True Negative = 225

Score Recall sudah sangat bagus, di atas 90%. akan tetapi, jika dilihat perbedaan antara train score dan test score, gap masih sangat tinggi dan termasuk overfitting. Selain itu, Presisi juga masih sangat rendah yang artinya False Positive masih sangat tinggi.  <b> Perlu dicoba lagi model algorithma lainnya. 

## C. Decision Tree

### Fit & Predict

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(random_state=42)
model.fit(X_over_SMOTE,y_over_SMOTE)

y_pred = model.predict(X_test)
eval_classification(model, y_pred, X_over_SMOTE, y_over_SMOTE, X_test, y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_pred, y_test)

- True positive = 1269
- False Negative = 45
- False Positive = 70
- True Negative = 221

<b> Recall_score sudah oke. Precision juga sudah cukup baik. Perlu dilakukan lagi hyperparameter tuning

### Tune Hyperparameter with RandomSearch
Setiap algoritma mempunyai list of hyperparameter yang berbeda-beda. lihat list hyperparameternya di dokumentasi sklearn.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
import numpy as np

# List of hyperparameter
max_depth = [int(x) for x in np.linspace(1, 110, num = 30)] # Maximum number of levels in tree
min_samples_split = [2, 5, 10, 100] # Minimum number of samples required to split a node
min_samples_leaf = [1, 2, 4, 10, 20, 50] # Minimum number of samples required at each leaf node
max_features = ['auto', 'sqrt'] # Number of features to consider at every split

hyperparameters = dict(max_depth=max_depth, 
                       min_samples_split=min_samples_split, 
                       min_samples_leaf=min_samples_leaf,
                       max_features=max_features
                      )

# Inisialisasi Model
dt = DecisionTreeClassifier(random_state=42)
model = RandomizedSearchCV(dt, hyperparameters, cv=5, random_state=42, scoring='recall')
model.fit(X_over_SMOTE, y_over_SMOTE)

# Predict & Evaluation
y_pred = model.predict(X_test)#Check performa dari model
eval_classification(model, y_pred, X_over_SMOTE, y_over_SMOTE, X_test, y_test)

In [ ]:
print('Best max_depth:', model.best_estimator_.get_params()['max_depth'])
print('Best min_samples_split:', model.best_estimator_.get_params()['min_samples_split'])
print('Best min_samples_leaf:', model.best_estimator_.get_params()['min_samples_leaf'])
print('Best max_features:', model.best_estimator_.get_params()['max_features'])

In [ ]:
from sklearn import tree
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(60, 20))
tree.plot_tree(model.best_estimator_,
               feature_names = X.columns.tolist(), 
               class_names=['0','1'],
               filled = True, max_depth=5, fontsize=10)
plt.show()

In [ ]:
print('Train score: ' + str(model.score(X_over_SMOTE, y_over_SMOTE))) #recall
print('Test score:' + str(model.score(X_test, y_test))) #recall

Overfitting

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_pred, y_test)

- True positive = 1242
- False Negative = 94
- False Positive = 97
- True Negative = 172

In [ ]:
# plt.figsize(10, 8)
feat_importances = pd.Series(model.best_estimator_.feature_importances_, index=X.columns)
ax = feat_importances.nlargest(25).plot(kind='barh', figsize=(10, 8))
ax.invert_yaxis()

plt.xlabel('score')
plt.ylabel('feature')
plt.title('feature importance score')

<b> Overfit setelah dilakukan hyperparameter. Trial dengan model algoritma lain. 

## D. Bagging: Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=42)
rf.fit(X_over_SMOTE,y_over_SMOTE)

y_pred = rf.predict(X_test)
eval_classification(rf, y_pred, X_over_SMOTE, y_over_SMOTE, X_test, y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_pred, y_test)

- True positive = 1309
- False Negative = 45
- False Positive = 30
- True Negative = 221

In [ ]:
show_feature_importance(rf)

<b> Very Good. 
Recall score sudah mencapai 0,82, precision  0,92 dan F1 score 0,86. model ini dapat digunakan sebagai Alternatif. 

### Hyperparameter tuning with RandomizedSearch

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

#List Hyperparameters yang akan diuji
hyperparameters = dict(
                       n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 20)], # Jumlah subtree 
                       bootstrap = [True], # Apakah pakai bootstrapping atau tidak
                       criterion = ['gini','entropy'],
                       max_depth = [int(x) for x in np.linspace(10, 110, num = 11)],  # Maximum kedalaman tree
                       min_samples_split = [int(x) for x in np.linspace(start = 2, stop = 10, num = 5)], # Jumlah minimum samples pada node agar boleh di split menjadi leaf baru
                       min_samples_leaf = [int(x) for x in np.linspace(start = 1, stop = 10, num = 5)], # Jumlah minimum samples pada leaf agar boleh terbentuk leaf baru
                       max_features = ['auto', 'sqrt', 'log2'], # Jumlah feature yg dipertimbangkan pada masing-masing split
                       n_jobs = [-1], # Core untuk parallel computation. -1 untuk menggunakan semua core
                      )

# Init
rf = RandomForestClassifier(random_state=42)
rf_tuned = RandomizedSearchCV(rf, hyperparameters, cv=5, random_state=42, scoring='recall')
rf_tuned.fit(X_over_SMOTE,y_over_SMOTE)

# Predict & Evaluation
y_pred = rf_tuned.predict(X_test)#Check performa dari model
eval_classification(rf_tuned, y_pred, X_over_SMOTE, y_over_SMOTE, X_test, y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_pred, y_test)

- True positive = 1307
- False Negative = 43
- False Positive = 32
- True Negative = 223

In [ ]:
show_best_hyperparameter(rf_tuned.best_estimator_, hyperparameters)

In [ ]:
show_feature_importance(rf_tuned.best_estimator_)

## E. Boosting: AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ab = AdaBoostClassifier(random_state=42)
ab.fit(X_over_SMOTE,y_over_SMOTE)

y_pred = ab.predict(X_test)
eval_classification(ab, y_pred, X_over_SMOTE, y_over_SMOTE, X_test, y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_pred, y_test)

- True positive = 1208
- False Negative = 54
- False Positive = 131
- True Negative = 212

In [ ]:
show_feature_importance(ab)

### Hyperparameter Tuning Adaboost

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import numpy as np

# List of hyperparameter
hyperparameters = dict(n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 20)], # Jumlah iterasi
                       learning_rate = [float(x) for x in np.linspace(start = 0.001, stop = 0.1, num = 20)],  
                       algorithm = ['SAMME', 'SAMME.R']
                      )

# Init model
ab = AdaBoostClassifier(random_state=42)
ab_tuned = RandomizedSearchCV(ab, hyperparameters, random_state=42, cv=5, scoring='recall')
ab_tuned.fit(X_over_SMOTE,y_over_SMOTE)

# Predict & Evaluation
y_pred = ab_tuned.predict(X_test)#Check performa dari model
eval_classification(ab_tuned, y_pred, X_over_SMOTE, y_over_SMOTE, X_test, y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_pred, y_test)

- True positive = 1230
- False Negative = 51
- False Positive = 109
- True Negative = 215

Perlu dioptimasi dengan model lain. 

In [ ]:
show_best_hyperparameter(ab_tuned.best_estimator_, hyperparameters)

In [ ]:
show_feature_importance(ab_tuned.best_estimator_)

## F. Boosting: XGBoost

In [ ]:
from xgboost import XGBClassifier
xg = XGBClassifier(random_state=42)
xg.fit(X_over_SMOTE, y_over_SMOTE)

y_pred = xg.predict(X_test)
eval_classification(xg, y_pred, X_over_SMOTE, y_over_SMOTE, X_test, y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_pred, y_test)

- True positive = 1248
- False Negative = 56
- False Positive = 91
- True Negative = 210

Perlu dioptimasi lagi dengan tuning hyperparameter

#### FEATURE IMPORTANCE

In [ ]:
# plt.figsize(10, 8)
feat_importances = pd.Series(model.best_estimator_.feature_importances_, index=X.columns)
ax = feat_importances.nlargest(25).plot(kind='barh', figsize=(10, 8))
ax.invert_yaxis()

plt.xlabel('score')
plt.ylabel('feature')
plt.title('feature importance score')

## Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import numpy as np

#Menjadikan ke dalam bentuk dictionary
hyperparameters = {
                    'max_depth' : [int(x) for x in np.linspace(10, 110, num = 11)],
                    'min_child_weight' : [int(x) for x in np.linspace(1, 20, num = 11)],
                    'gamma' : [float(x) for x in np.linspace(0, 1, num = 11)],
                    'tree_method' : ['auto', 'exact', 'approx', 'hist'],

                    'colsample_bytree' : [float(x) for x in np.linspace(0, 1, num = 11)],
                    'eta' : [float(x) for x in np.linspace(0, 1, num = 100)],

                    'lambda' : [float(x) for x in np.linspace(0, 1, num = 11)],
                    'alpha' : [float(x) for x in np.linspace(0, 1, num = 11)]
                    }

# Init
xg = XGBClassifier(random_state=42)
xg_tuned = RandomizedSearchCV(xg, hyperparameters, cv=5, random_state=42, scoring='recall')
xg_tuned.fit(X_over_SMOTE,y_over_SMOTE)

# Predict & Evaluation
y_pred = xg_tuned.predict(X_test)#Check performa dari model
eval_classification(xg_tuned, y_pred, X_over_SMOTE, y_over_SMOTE, X_test, y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_pred, y_test)

- True positive = 1310
- False Negative = 38
- False Positive = 29
- True Negative = 228

Model ini sudah sangat baik. dengan score Recall yang sudah cukup optimum dan score precision yang juga sangat baik. Paduan score ini menjadi XGBoost model sebagai model machine learning yang akan digunakan dalam bisnis. Dengan prinsip FN dan FP sekecilnya-kecilnya. Dengan demikian, retensi pelanggan dapat ditingkatkan sesuai sasaran. 
<b> MODEL INI SUDAH BEST FIT. 

In [ ]:
show_best_hyperparameter(xg_tuned.best_estimator_, hyperparameters)

In [ ]:
show_feature_importance(xg_tuned.best_estimator_)

# TOP FEATURE

In [ ]:
df_retrain =df_z2.copy()
df_retrain.info()

In [ ]:
# pemisahan features vs target
X = df_retrain[[col for col in df_retrain.columns if (str(df_retrain[col].dtype) != 'object') and col not in ['churn_class', 'Churn', 'PreferedOrderCat_Others']]]
y = df_retrain['churn_class'].values
print(X.shape)
print(y.shape)

In [ ]:
#Splitting the data into Train & Test
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, random_state = 42)

In [ ]:
from imblearn import under_sampling, over_sampling
X_under, y_under = under_sampling.RandomUnderSampler(1).fit_resample(X_train, y_train)
X_over, y_over = over_sampling.RandomOverSampler(1).fit_resample(X_train, y_train)
X_over_SMOTE, y_over_SMOTE = over_sampling.SMOTE(1).fit_resample(X_train, y_train)

## XGBOOST

In [ ]:
from xgboost import XGBClassifier
xg = XGBClassifier(random_state=42)
xg.fit(X_over_SMOTE, y_over_SMOTE)

y_pred = xg.predict(X_test)
eval_classification(xg, y_pred, X_over_SMOTE, y_over_SMOTE, X_test, y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_pred, y_test)

### HYPERPARAMETER TUNING

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import numpy as np

#Menjadikan ke dalam bentuk dictionary
hyperparameters = {
                    'max_depth' : [int(x) for x in np.linspace(10, 110, num = 11)],
                    'min_child_weight' : [int(x) for x in np.linspace(1, 20, num = 11)],
                    'gamma' : [float(x) for x in np.linspace(0, 1, num = 11)],
                    'tree_method' : ['auto', 'exact', 'approx', 'hist'],

                    'colsample_bytree' : [float(x) for x in np.linspace(0, 1, num = 11)],
                    'eta' : [float(x) for x in np.linspace(0, 1, num = 100)],

                    'lambda' : [float(x) for x in np.linspace(0, 1, num = 11)],
                    'alpha' : [float(x) for x in np.linspace(0, 1, num = 11)]
                    }

# Init
xg = XGBClassifier(random_state=42)
xg_tuned = RandomizedSearchCV(xg, hyperparameters, cv=5, random_state=42, scoring='recall')
xg_tuned.fit(X_over_SMOTE,y_over_SMOTE)

# Predict & Evaluation
y_pred = xg_tuned.predict(X_test)#Check performa dari model
eval_classification(xg_tuned, y_pred, X_over_SMOTE, y_over_SMOTE, X_test, y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_pred, y_test)

# SIMULASI

## A. SIMULASI KETIKA FITUR GAINING POIN/COIN TELAH BERJALAN SELAMA 6 BULAN

Fitur gaining point/coin adalah fitur yang dimaksudkan dan diutamakan untuk customer dengan tenure < 2 bulan atau customer yang baru saja diakusisi. berdasarkan data, pemberian cashback saja belum bisa meningkatkan retensi dari new user ini. oleh karena itu dibuat fitur baru berupa gaining point yang diperoleh dari bermain game 'marvel farming' misalnya. dan skema kedua, gaining point berdasarkan jumlah produk yang telah diorder. Dengan adanya fitur ini, diharapkan dapat meningkatkan retensi dari customer. 
Asumsi fitur ini sudah berjalan selama 6 bulan. maka skenarionya sebagai berikut :

In [ ]:
df_sim = df.copy()

In [ ]:
df_sim.head()

#### <B> HAPUS OUTLIER

In [ ]:
from scipy import stats

print(f'Jumlah baris sebelum memfilter outlier: {len(df_sim)}')

filtered_entries = np.array([True] * len(df_sim))

for col in df[nums]:
    zscore = abs(stats.zscore(df_sim[col])) # hitung absolute z-scorenya
    filtered_entries = (zscore < 3) & filtered_entries # keep yang kurang dari 3 absolute z-scorenya
    
df_sim2 = df_sim[filtered_entries] # filter, cuma ambil yang z-scorenya dibawah 3

print(f'Jumlah baris setelah memfilter outlier: {len(df_sim2)}')
print('Maka outlier yang dihapus sekitar', round((len(df_sim)-len(df_sim2))/len(df_sim)*100), '%')

In [ ]:
df_simulation = df_sim2.copy()

ada 1182 customer yang berstatus tenure = 0 bulan dan tenure =1 bulan. 

### <B> MANIPULASI TENURE DI ATAS 2 BULAN DENGAN MENAIKKAN TENURE HINGGA +6 BULAN

In [ ]:
df_sim3 = df_sim2.copy()

<B> Asumsi bahwa customer dengan Tenure > 2 bulan sudah setia menggunakan e-commerce kita selama 6 bulan ke depan. dan diasumsi bahwa data yang kita miliki sekarang ini adalah keadaan customer setelah 6 bulan menggunakan fitur gaining point yang kita buat. 

In [ ]:
df_sim3['Tenure'].replace(31, 37, inplace = True)
df_sim3['Tenure'].replace(30, 36, inplace = True)
df_sim3['Tenure'].replace(29, 35, inplace = True)
df_sim3['Tenure'].replace(28, 34, inplace = True)
df_sim3['Tenure'].replace(27, 33, inplace = True)
df_sim3['Tenure'].replace(26, 32, inplace = True)
df_sim3['Tenure'].replace(25, 31, inplace = True)
df_sim3['Tenure'].replace(24, 30, inplace = True)
df_sim3['Tenure'].replace(23, 29, inplace = True)
df_sim3['Tenure'].replace(22, 28, inplace = True)
df_sim3['Tenure'].replace(21, 27, inplace = True)
df_sim3['Tenure'].replace(20, 26, inplace = True)
df_sim3['Tenure'].replace(19, 25, inplace = True)
df_sim3['Tenure'].replace(18, 24, inplace = True)
df_sim3['Tenure'].replace(17, 23, inplace = True)
df_sim3['Tenure'].replace(16, 22, inplace = True)
df_sim3['Tenure'].replace(15, 21, inplace = True)
df_sim3['Tenure'].replace(14, 20, inplace = True)
df_sim3['Tenure'].replace(13, 19, inplace = True)
df_sim3['Tenure'].replace(12, 18, inplace = True)
df_sim3['Tenure'].replace(11, 17, inplace = True)
df_sim3['Tenure'].replace(10, 16, inplace = True)
df_sim3['Tenure'].replace(9, 15, inplace = True)
df_sim3['Tenure'].replace(8, 14, inplace = True)
df_sim3['Tenure'].replace(7, 13, inplace = True)
df_sim3['Tenure'].replace(6, 12, inplace = True)
df_sim3['Tenure'].replace(5, 11, inplace = True)
df_sim3['Tenure'].replace(4, 10, inplace = True)
df_sim3['Tenure'].replace(3, 9, inplace = True)
df_sim3['Tenure'].replace(2, 7, inplace = True)

In [ ]:
df_sim2.head()

In [ ]:
df_sim3.head(5)

In [ ]:
df_sim4 = df_sim3.copy()

In [ ]:
df_0_1 = df_sim4[df_sim4['Tenure']== 0]
df_0_1.groupby('OrderCount').agg({'CustomerID' : 'count'})

In [ ]:
df_0_1 = df_sim4[df_sim4['Tenure'] == 1]
df_0_1.groupby('OrderCount').agg({'CustomerID' : 'count'})

In [ ]:
df_0_1ten = df_sim4[(
    (df_sim4['Tenure'] == 0) | (df_sim4['Tenure'] == 1) 
    )]
df_0_1ten

untuk customer dnegan Tenure < 2 bulan (0 dan 1 bulan) akan dilakukan clustering untuk menentukan atau memberikan asumsi seberapa lama tenure nya dapat ditingkatkan. 

In [ ]:
df_asumsi_5 = df_sim3.copy()

In [ ]:
df_asumsi_t1 = df_sim3.copy()

<b> Beberappa fitur yang berkorelasi dengan `Tenure` : 
- Number of Address
- Day Since Last Order
- CashbackAmount
- OrderCount

### <B> UNTUK TENURE = 0 BULAN

In [ ]:
df_asumsi_5 = df_asumsi_5[df_asumsi_5['Tenure']<1]

In [ ]:
df_asumsi_5a = df_asumsi_5.copy()

In [ ]:
df_asumsi_5a = df_asumsi_5a[['Churn', 'NumberOfAddress', 'DaySinceLastOrder', 'CashbackAmount', 'OrderCount']]
df_asumsi_5a.head()

In [ ]:
from sklearn.preprocessing import StandardScaler


df_asumsi_5a['NumberOfAddress'] = StandardScaler().fit_transform(df_asumsi_5a['NumberOfAddress'].values.reshape(len(df_asumsi_5a), 1))
df_asumsi_5a['DaySinceLastOrder'] = StandardScaler().fit_transform(df_asumsi_5a['DaySinceLastOrder'].values.reshape(len(df_asumsi_5a), 1))
df_asumsi_5a['CashbackAmount'] = StandardScaler().fit_transform(df_asumsi_5a['CashbackAmount'].values.reshape(len(df_asumsi_5a), 1))
df_asumsi_5a['OrderCount'] = StandardScaler().fit_transform(df_asumsi_5a['OrderCount'].values.reshape(len(df_asumsi_5a), 1))

df_asumsi_5a.head()

In [ ]:
corr_= df_asumsi_5a.corr()
plt.figure(figsize=(16,10))
sns.heatmap(corr_, annot=True, fmt = ".2f", cmap = "BuPu")

#### MENENTUKAN  JUMLAH CLUSTER BEDASARKAN ELBOW METHOD

In [ ]:
from sklearn.cluster import KMeans
inertia = []

for i in range(2, 11):
    kmeans = KMeans(n_clusters=i, random_state=0)
    kmeans.fit(df_asumsi_5a.values)
    inertia.append(kmeans.inertia_)


print(inertia)

plt.figure(figsize=(20, 10))
# plt.plot(inertia)

sns.lineplot(x=range(2, 11), y=inertia, color='#000087', linewidth = 4)
sns.scatterplot(x=range(2, 11), y=inertia, s=300, color='#800000',  linestyle='--');

#### CLUSTERING DENGAN K-MEANS

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=6, random_state=42)
kmeans.fit(df_asumsi_5a.values)

In [ ]:
df_asumsi_5a['cluster'] = kmeans.labels_
df_asumsi_5a.head()

Dari cluster yang telah ditentukan, diasumsikan bahwa cluster 0 adalah customer yang tidak akan melanjutkan untuk menggunakan layanan e-commerce kita atau tenure = 0. sedangkan 1, 2, 3, 4, 5, 6 melambangkan Tenure mereka. 

#### MENEMPELKAN LABEL 

In [ ]:
df_asumsi_label = df_asumsi_5

In [ ]:
df_asumsi_label['cluster'] = kmeans.labels_
df_asumsi_label.head()

### <B> BERDASARKAN TENURE = 1 BULAN

In [ ]:
df_asumsi_t1 = df_asumsi_t1[df_asumsi_t1['Tenure']== 1]

In [ ]:
df_1a = df_asumsi_t1.copy()

In [ ]:
df_1a = df_1a[['Churn', 'NumberOfAddress', 'DaySinceLastOrder', 'CashbackAmount', 'OrderCount']]
df_1a.head()

In [ ]:
from sklearn.preprocessing import StandardScaler


df_1a['NumberOfAddress'] = StandardScaler().fit_transform(df_1a['NumberOfAddress'].values.reshape(len(df_1a), 1))
df_1a['DaySinceLastOrder'] = StandardScaler().fit_transform(df_1a['DaySinceLastOrder'].values.reshape(len(df_1a), 1))
df_1a['CashbackAmount'] = StandardScaler().fit_transform(df_1a['CashbackAmount'].values.reshape(len(df_1a), 1))
df_1a['OrderCount'] = StandardScaler().fit_transform(df_1a['OrderCount'].values.reshape(len(df_1a), 1))

df_1a.head()

In [ ]:
corr_= df_1a.corr()
plt.figure(figsize=(16,10))
sns.heatmap(corr_, annot=True, fmt = ".2f", cmap = "BuPu")

#### MENENTUKAN JUMLAH CLUSTER DENGAN ELBOW METHOD

In [ ]:
from sklearn.cluster import KMeans
inertia = []

for i in range(2, 11):
    kmeans = KMeans(n_clusters=i, random_state=0)
    kmeans.fit(df_1a.values)
    inertia.append(kmeans.inertia_)


print(inertia)

plt.figure(figsize=(20, 10))
# plt.plot(inertia)

sns.lineplot(x=range(2, 11), y=inertia, color='#000087', linewidth = 4)
sns.scatterplot(x=range(2, 11), y=inertia, s=300, color='#800000',  linestyle='--');

#### CLUSTERING DENGAN K-MEANS

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=6, random_state=42)
kmeans.fit(df_1a.values)

In [ ]:
df_1a['cluster'] = kmeans.labels_
df_1a.head()

#### LABELLING

In [ ]:
df_asume_label = df_asumsi_t1.copy()

In [ ]:
df_asume_label['cluster'] = kmeans.labels_
df_asume_label.head()

### <B> IMPUTASI NILAI CLUSTER TADI SEBAGAI TENURE

#### <B> UNTUK TENURE = 0 BULAN

In [ ]:
df_simulation.head()

In [ ]:
df_trial = df_simulation.copy()

In [ ]:
df_adi = df_trial[~(df_trial['Tenure']==0)]

In [ ]:
df_adi1 = df_adi[~(df_adi['Tenure']==1)]
df_adi1

#### <B> UNTUK TENURE = 1 BULAN

Asumsi Tenure 1 bulan akan berubah menjadi :    
- 1 ---> ketika cluster = 5
- 2 ---> ketika cluster = 4
- 3 ---> ketika cluster = 3
- 4 ---> ketika cluster = 2
- 5 ---> ketika cluster = 1
- 6 ---> ketika cluster = 0

In [ ]:
df_asume_label['Tenure'] = abs(df_asume_label['Tenure'] - 2*df_asume_label['Tenure']-df_asume_label['cluster'])
df_asume_label

#### <B> MENGGABUNGKAN CUSTOMER DENGAN TENURE 0 DAN 1 YANG SUDAH DIMANIPULASI BERDASARKAN CLUSTERING

In [ ]:
df_trial_oke = df_simulation.copy()

In [ ]:
df_ten_0_1 = df_asumsi_label.merge(df_asume_label, how = 'outer')
df_ten_0_1

In [ ]:
df_ten_0_1['Tenure'] = abs(df_ten_0_1['Tenure'] - df_ten_0_1['cluster'])
df_ten_0_1

In [ ]:
df_adi_oke = df_ten_0_1.drop(['cluster'], axis = 1)
df_adi_oke

#### <B> MENGGABUNGKAN DATA YANG SUDAH DIMANIPULASI. DATASET READY !

In [ ]:
df_merge = df_adi1.merge(df_adi_oke, how = 'outer')
df_merge

In [ ]:
df_merge.duplicated().sum()

<b> Jumlah baris dataset yang sudah dimanipulasi berjumlah 5350. Semua sebaran tenure sudah disesuaikan secara merata untuk tenure 0 bulan dan 1 bulan. 

### MENGETAHUI KENAIKAN TENURE SETELAH 6 BULAN

In [ ]:
df_z['Tenure'].mean()

In [ ]:
df_merge['Tenure'].sum()

In [ ]:
print('Maka,kenaikan tenure nya selama fitur gaining point berlangsung adalah', (df_merge['Tenure'].sum() - df_z['Tenure'].sum())/df_z['Tenure'].sum()*100, '%')

### <B> PENYESUAIAN HOURSPEND ON APP BERDASARKAN TENURE

Asumsi bahwa : rata-rata customer membuka aplikasi/website untuk menggunakan fitur gaining poin kita selama 5 menit perhari. Maka, Hourspend on App stelah 6 bulan adalah = `Tenure * 30 * 0.08* dalam menit. 

In [ ]:
df_merge['HourSpendOnApp'] = df_merge['HourSpendOnApp']+ 30*0.08
df_merge

## HOURSPEND ON APP SETELAH ADANYA FITUR GAINING POINT : 

In [ ]:
df_z['HourSpendOnApp'].sum()

In [ ]:
df_merge['HourSpendOnApp'].sum()

### <B> MANIPULASI COMPLAIN

In [ ]:
df_merge[df_merge['Complain'] == 1]

In [ ]:
data_complains

Dari data dapat dilihat bahwa jumlah customer yang complain adalah 1532 orang, dengan :
- jumlah yang churn dari customer yang complain = 1036
- jumlah yang tidak churn dari customer yang complain = 496 


Asumsi bahwa :     
- Resolved complain mencapai 100% hingga hari ini. 

In [ ]:
df_merge['Complain'].replace(1, 0, inplace = True)

## B. PREDIKSI SATISFACTION SCORE

#### Split Train & Test

In [ ]:
df_model2 = df_z2.copy()

In [ ]:
#melihat kembali korelasi fitur terhadap label (churn)
plt.figure(figsize = (30, 26))
heatmaps=sns.heatmap(df_z2.corr(),cmap='Blues', annot=True, fmt='.2f')
heatmaps.set_title('Korelasi', fontdict={'fontsize':18}, pad=16)

In [ ]:
df_model2.info()

In [ ]:
# pemisahan features vs target
X = df_model2[[col for col in df_model2.columns if (str(df_model2[col].dtype) != 'object') and col not in ['churn_class', 'CityTier', 'HourSpendOnApp', 'CashbackAmount_normalisasi', 'NumberOfDeviceRegistered_normalisasi',  'DaySinceLastOrder_normalisasi', 'PreferedOrderCat_Others', 'SatisfactionScore', 'std_Tenure', 'std_WarehouseToHome', 'MaritalStatus_Single', 
                                                                                                        'PreferedOrderCat_Fashion',  'PreferedOrderCat_Grocery', 'PreferedOrderCat_Laptop & Accessory', 'PreferedOrderCat_Mobile Phone',  'CustomerCategory']]]
y = df_model2['SatisfactionScore'].values
print(X.shape)
print(y.shape)

In [ ]:
#Splitting the data into Train & Test
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 42)

In [ ]:
import imblearn
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 101)
X_oversample, y_oversample = smote.fit_resample(X_train, y_train)

In [ ]:
plt.figure(figsize=(10,8))
sns.pairplot(data=df_model2, x_vars=df_model2[[col for col in df_model2.columns if (str(df_model2[col].dtype) != 'object') and col not in ['SatisfactionScore']]],  y_vars=['SatisfactionScore'], size=15, aspect=0.75)

#### Fit Model 

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
def eval_regression(model, pred, X_train, y_train, X_test, y_test):
    print("MAE: %.2f" % mean_absolute_error(y_test, pred)) # The MAE
    print("RMSE: %.2f" % mean_squared_error(y_test, pred, squared=False)) # The RMSE
    print('R2 score: %.2f' % r2_score(y_test, pred)) 

#### 1. DECISION TREE REGRESSOR

In [ ]:
from sklearn.tree import DecisionTreeRegressor

dt = DecisionTreeRegressor()
dt.fit( X_oversample, y_oversample)
pred = dt.predict(X_test)
eval_regression(dt, pred, X_oversample, y_oversample, X_test, y_test)

In [ ]:
print("train Accuracy : ",dt.score( X_oversample, y_oversample))
print("test Accuracy : ",dt.score(X_test,y_test))

#### 2. RANDOM FORES REGRESSOR

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()
rf.fit( X_oversample, y_oversample)
pred = rf.predict(X_test)
eval_regression(rf, pred,  X_oversample, y_oversample, X_test, y_test)

In [ ]:
print("train Accuracy : ",rf.score( X_oversample, y_oversample))
print("test Accuracy : ",rf.score(X_test,y_test))

#### SVR MODEL

In [ ]:
from sklearn.svm import SVR

svr = SVR()
svr.fit( X_oversample, y_oversample)
pred = svr.predict(X_test)
eval_regression(svr, pred, X_oversample, y_oversample, X_test, y_test)

In [ ]:
print("train Accuracy : ",svr.score( X_oversample, y_oversample))
print("test Accuracy : ",svr.score(X_test,y_test))

In [ ]:
# RIDGE
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from scipy.stats import uniform

alpha = [0.01, 0.1, 1, 2, 5, 10, 100, 200, 230, 250, 265, 270, 275, 290, 300, 500] # alpha
solver = ['lsqr', 'auto', 'sag', 'saga']
hyperparameters = dict(alpha=alpha, solver=solver)

from sklearn.linear_model import Ridge
ridge_model = Ridge()
clf = RandomizedSearchCV(ridge_model, hyperparameters, cv=5, random_state=42, scoring='r2')

#Fitting Model
best_model = clf.fit( X_oversample, y_oversample)
pred = best_model.predict(X_test)
eval_regression(best_model, pred,  X_oversample, y_oversample, X_test, y_test)

In [ ]:
print('Best solver:', best_model.best_estimator_.get_params()['solver'])
print('Best alpha:', best_model.best_estimator_.get_params()['alpha'])

In [ ]:
print("train Accuracy : ",clf.score( X_oversample, y_oversample))
print("test Accuracy : ",clf.score(X_test,y_test))

<b> KARENA TIDAK ADA FITUR YANG BERKORELASI KUAT DENGAN `SatifactionScore`, DAN HAMPIR SEMUA FITUR NYA TIDAK BERKOREALSI THD `SatifactionScore` INI, MAKA DAPAT DISIMPULKAN BAHWA `SatifactionScore` TIDAK DAPAT DISIMPULKAN. 

## C. PRE-PROCESSING UNTUK MODELLING

#### FEATURE TRANSFORMATION

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler


df_merge['CashbackAmount_normalisasi'] = MinMaxScaler().fit_transform(df_merge['CashbackAmount'].values.reshape(len(df_merge), 1))
df_merge['NumberOfDeviceRegistered_normalisasi'] = MinMaxScaler().fit_transform(df_merge['NumberOfDeviceRegistered'].values.reshape(len(df_merge), 1))
df_merge['DaySinceLastOrder_normalisasi'] = MinMaxScaler().fit_transform(df_merge['DaySinceLastOrder'].values.reshape(len(df_merge), 1))

df_merge['std_Tenure'] = StandardScaler().fit_transform(df_merge['Tenure'].values.reshape(len(df_merge), 1))
df_merge['std_WarehouseToHome'] = StandardScaler().fit_transform(df_merge['WarehouseToHome'].values.reshape(len(df_merge), 1))


df_merge

<B> MENGUBAH VALUE YANG TIDAK KONSISTEN PADA TIPE DATA CATEGORICAL 

In [ ]:
df_merge['PreferredLoginDevice'].replace('Phone', 'Mobile Phone', inplace = True)
df_merge['PreferredPaymentMode'].replace('CC', 'Credit Card', inplace = True)
df_merge['PreferredPaymentMode'].replace('COD', 'Cash on Delivery', inplace = True)
df_merge['PreferedOrderCat'].replace('Mobile', 'Mobile Phone', inplace = True)

<B> FEATURE ENCODING

In [ ]:
# Mapping Gender
mapping_gender = {
    'Female' : 0,
    'Male' : 1
}
df_merge['Gender'] = df_merge['Gender'].map(mapping_gender)

In [ ]:
for cat in ['MaritalStatus', 'PreferredLoginDevice', 'PreferedOrderCat']:
    onehots = pd.get_dummies(df_merge[cat], prefix=cat)
    df_merge = df_merge.join(onehots)
#PreferedPaymentMethod tidak perlu di feature encoding. pengaruh feature ini terhadap label sangat kecil

<B> FEATURE SELECTION

In [ ]:
# drop kolom2 yang asli (karena sudah di-encoding)
df_merge= df_merge.drop(columns=['PreferredLoginDevice', 'MaritalStatus','PreferredPaymentMode','PreferedOrderCat', 'CustomerID', 'Tenure','NumberOfDeviceRegistered', 'WarehouseToHome',
                        'DaySinceLastOrder', 'CashbackAmount', 'OrderAmountHikeFromlastYear', 'CouponUsed', 'Gender' ])


#### FEATURE EXTRACTION

In [ ]:
df_asumsi_av = df_merge.copy()

In [ ]:
klasifikasi = []

for i, kolom in df_asumsi_av.iterrows():
    if kolom ['OrderCount'] < 4: 
        result = 'Bronze'
    elif kolom ['OrderCount'] < 9 and kolom ['OrderCount'] >=8 : 
        result = 'Silver'
    else :
        result = 'Gold'
    klasifikasi.append(result)
df_asumsi_av['CustomerCategory']=klasifikasi
df_asumsi_av

In [ ]:
# Mapping CustomerCategory
mapping_CustomerCategory = {
    'Bronze' : 0,
    'Silver' : 1,
    'Gold' : 2
}
df_asumsi_av['CustomerCategory'] = df_asumsi_av['CustomerCategory'].map(mapping_CustomerCategory)

In [ ]:
# drop kolom2 yang asli (karena sudah di-encoding)
df_asumsi_av= df_asumsi_av.drop(columns=['OrderCount' ])
df_asumsi_av.head()


### MODELLING

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, auc

def eval_classification(model, pred, xtrain, ytrain, xtest, ytest):
    print("Accuracy (Test Set): %.2f" % accuracy_score(ytest, pred))
    print("Precision (Test Set): %.2f" % precision_score(ytest, pred))
    print("Recall (Test Set): %.2f" % recall_score(ytest, pred))
    print("F1-Score (Test Set): %.2f" % f1_score(ytest, pred))
    
    fpr, tpr, thresholds = roc_curve(ytest, pred, pos_label=1) # pos_label: label yang kita anggap positive
    print("AUC: %.2f" % auc(fpr, tpr))

def show_feature_importance(model):
    feat_importances = pd.Series(model.feature_importances_, index=X.columns)
    ax = feat_importances.nlargest(25).plot(kind='barh', figsize=(10, 8))
    ax.invert_yaxis()

    plt.xlabel('score')
    plt.ylabel('feature')
    plt.title('feature importance score')

def show_best_hyperparameter(model, hyperparameters):
    for key, value in hyperparameters.items() :
        print('Best '+key+':', model.get_params()[key])

In [ ]:
df_avengers = df_asumsi_av.copy()

In [ ]:
# pembuatan binary label target yang imbalance (sebagai contoh)
df_avengers['churn_class'] = df_avengers['Churn'] > 0.8
df_avengers['churn_class'].value_counts()

In [ ]:
X= df_avengers[[col for col in df_avengers.columns if (str(df_avengers[col].dtype) != 'object') and col not in ['Churn', 'churn_class','PreferedOrderCat_Others']]]
y = df_avengers['churn_class'].values
print(X.shape)
print(y.shape)

### XGBOOST

In [ ]:
from xgboost import XGBClassifier
xg = XGBClassifier(random_state=42)
xg.fit(X_over_SMOTE, y_over_SMOTE)

y_pred = xg.predict(X)
eval_classification(xg, y_pred, X_over_SMOTE, y_over_SMOTE, X, y)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_pred, y)

<b> HYPERPARAMETER TUNING

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import numpy as np

#Menjadikan ke dalam bentuk dictionary
hyperparameters = {
                    'max_depth' : [int(x) for x in np.linspace(10, 110, num = 11)],
                    'min_child_weight' : [int(x) for x in np.linspace(1, 20, num = 11)],
                    'gamma' : [float(x) for x in np.linspace(0, 1, num = 11)],
                    'tree_method' : ['auto', 'exact', 'approx', 'hist'],

                    'colsample_bytree' : [float(x) for x in np.linspace(0, 1, num = 11)],
                    'eta' : [float(x) for x in np.linspace(0, 1, num = 100)],

                    'lambda' : [float(x) for x in np.linspace(0, 1, num = 11)],
                    'alpha' : [float(x) for x in np.linspace(0, 1, num = 11)]
                    }

# Init
xg = XGBClassifier(random_state=42)
xg_tuned = RandomizedSearchCV(xg, hyperparameters, cv=5, random_state=42, scoring='recall')
xg_tuned.fit(X_over_SMOTE,y_over_SMOTE)

# Predict & Evaluation
y_pred = xg_tuned.predict(X)#Check performa dari model
eval_classification(xg_tuned, y_pred, X_over_SMOTE, y_over_SMOTE, X, y)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_pred, y)

# KESIMPULAN DARI MODEL :


- Model yang sudah best fit adalah XGBOOST dengan tuning hyperparameter. score Precision (Test Set): 0.90 dan Recall (Test Set): 0.86, dan F1 SCORE = 0.88. Model ini dapat digunakan sebagai model machine learning untuk bisnis kita. Dengan prinsip FN dan FP sekecilnya-kecilnya. Dengan demikian, retensi pelanggan dapat ditingkatkan dan operational cost untuk tujuan tersebut sesuai sasaran.

# Business Insight & Recommendation

- Karena `Tenure` merupakan feature terpenting dalam model, dan juga dilihat dari churn rate tertinggi ada di customer yg tenure nya rendah jadi bisa menggunakan stategi membuat fitur baru dalam layanan e-commerce seperti 'Gaining Poin or Coin' atau game mengumpulkan coin/poin tersebut yang nantinya dapat ditukar untuk meningkatkan rasa penasaran, experience dan ambisi dari customer sehingga dapat meningkatkan retensi dari pelanggan. Point/coin ini nantinya dapat ditukar dengan voucher gratis ongkir contohnya, atau cashback dan lain-lain. 
- Semakin banyak jumlah pesanan/order, semakin banyak kupon yang digunakan oleh customer. jadi, jumlah order produk itu sangat ditentukan oleh kupon yang kita berikan, oleh karena itu kita harus tetap memberikan kupon kepada customer dengan mempertimbangkan ROI, BEP dll agar customer tetap setia dan rutin dalam melakukan pemesanan produk.
- Mayoritas dari customer yang menggunakan cashback adalah customer dengan lama menggunakan layanan kita kurang dari 21 bulan. dan customer yang sudah memperoleh cashback lebih cenderung churn. fenomena ini bisa menjadi pertimbangan dalam pemberian cashback kepada pelanggan. Sungguh disayangkan kita sudah mengalirkan dana operasional berupa cashback kepada pelanggan, dan pelanggan justru churn, karena pelanggan hanya ingin mengambil cashback selama melakukan pembelian di layanan e commerce kita. Terlihat jelas bahwa customer yang melakukan order hanya 1 kali, juga paling banyak menggunakan cashback. Kemungkinan memang customer ini hanya ingin mengambil keutungan berupa cashback yang besar sebagai new user. Untuk melakukan akusisi pelanggan baru, juga perlu mempertimbangkan bagaimana cara meningkatkan retensi mereka. Perlu adanya fitur farming/game yang hasilnya dapat ditukarkan dengan syarat, atau pengumpulan poin dari hasil pembelian produk dan lain-lain. dengan demikian, hal ini dapat meningkatkan retensi dan loyalitas dari new customer.
- Selain itu, kita juga bisa memberikan rekomendasi bisnis dengan meningkatkan penawaran produk ecommerce pada customer sesuai usia, gender, dll., serta membuat fast call service untuk penanganan masalah produk dengan cepat yang bertujuan untuk retensi pelanggan.
